In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

import sklearn
from sklearn import model_selection as ms
from sklearn.linear_model import LogisticRegression

# sns.set(rc={'figure.figsize':(11.7,8.27)})

import gzip

import copy

import import_ipynb
from My_Functions import null_cols, rmse

importing Jupyter notebook from My_Functions.ipynb


In [2]:
acc = pd.read_csv('../../accepted_2007_to_2018Q4.csv.gz',
                  compression='gzip',
                  header=0,
                  sep=',',
                  quotechar='"')
acc

C:\Users\tdcho\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (0,19,49,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,leadman,10+ years,MORTGAGE,55000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,190xx,PA,5.91,0.0,Aug-2003,675.0,679.0,1.0,30.0,NaN,7.0,0.0,2765.0,29.7,13.0,w,0.00,0.00,4421.723917,4421.72,3600.00,821.72,0.00,0.0,0.0,Jan-2019,122.67,NaN,Mar-2019,564.0,560.0,0.0,30.0,1.0,Individual,NaN,NaN,NaN,0.0,722.0,144904.0,2.0,2.0,0.0,1.0,21.0,4981.0,36.0,3.0,3.0,722.0,34.0,9300.0,3.0,1.0,4.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,0.0,0.0,0.0,3.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,Engineer,10+ years,MORTGAGE,65000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,small_business,Business,577xx,SD,16.06,1.0,Dec-1999,715.0,719.0,4.0,6.0,NaN,22.0,0.0,21470.0,19.2,38.0,w,0.00,0.00,25679.660000,25679.66,24700.00,979.66,0.00,0.0,0.0,Jun-2016,926.35,NaN,Mar-2019,699.0,695.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,204396.0,1.0,1.0,0.0,1.0,19.0,18005.0,73.0,2.0,3.0,6472.0,29.0,111800.0,0.0,0.0,6.0,4.0,9733.0,57830.0,27.1,0.0,0.0,113.0,192.0,2.0,2.0,4.0,2.0,NaN,0.0,6.0,0.0,5.0,5.0,13.0,17.0,6.0,20.0,27.0,5.0,22.0,0.0,0.0,0.0,2.0,97.4,7.7,0.0,0.0,314017.0,39475.0,79300.0,24667.0,NaN,NaN,NaN,NaN,NaN

In [3]:
# we will remove those rows
rows_to_drop_id = list(acc[acc['id'].str.contains('Total|Loans', na=False)].index)

acc = acc.drop(rows_to_drop_id)
acc

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,leadman,10+ years,MORTGAGE,55000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,190xx,PA,5.91,0.0,Aug-2003,675.0,679.0,1.0,30.0,NaN,7.0,0.0,2765.0,29.7,13.0,w,0.00,0.00,4421.723917,4421.72,3600.00,821.72,0.00,0.0,0.0,Jan-2019,122.67,NaN,Mar-2019,564.0,560.0,0.0,30.0,1.0,Individual,NaN,NaN,NaN,0.0,722.0,144904.0,2.0,2.0,0.0,1.0,21.0,4981.0,36.0,3.0,3.0,722.0,34.0,9300.0,3.0,1.0,4.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,0.0,0.0,0.0,3.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,Engineer,10+ years,MORTGAGE,65000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,small_business,Business,577xx,SD,16.06,1.0,Dec-1999,715.0,719.0,4.0,6.0,NaN,22.0,0.0,21470.0,19.2,38.0,w,0.00,0.00,25679.660000,25679.66,24700.00,979.66,0.00,0.0,0.0,Jun-2016,926.35,NaN,Mar-2019,699.0,695.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,204396.0,1.0,1.0,0.0,1.0,19.0,18005.0,73.0,2.0,3.0,6472.0,29.0,111800.0,0.0,0.0,6.0,4.0,9733.0,57830.0,27.1,0.0,0.0,113.0,192.0,2.0,2.0,4.0,2.0,NaN,0.0,6.0,0.0,5.0,5.0,13.0,17.0,6.0,20.0,27.0,5.0,22.0,0.0,0.0,0.0,2.0,97.4,7.7,0.0,0.0,314017.0,39475.0,79300.0,24667.0,NaN,NaN,NaN,NaN,NaN

In [4]:
# Adding ROI to the dataset
acc['roi'] = (acc['total_pymnt'] - acc['funded_amnt'])/acc['funded_amnt']

acc['roi']

0          0.228257
1          0.039662
2          0.135296
3         -0.101028
4          0.128894
             ...   
2260694   -0.343313
2260695   -0.374645
2260696   -0.377402
2260697   -0.718525
2260698   -0.312768
Name: roi, Length: 2260668, dtype: float64

In [5]:
# Adding column for 70% of principal
acc[['principal_70']] = acc[['funded_amnt']] * .7

In [6]:
# dropping letters and converting strings to numeric for 'term' column
acc.loc[acc['term'].str.contains('36', na=False), 'term'] = 36
acc.loc[acc['term'].str.contains('60', na=False), 'term'] = 60

# converting the 'term' column to integer
acc['term'] = acc['term'].astype(int)

acc[['term']]

,term
0,36
1,36
2,60
3,60
4,60
...,...
2260694,60
2260695,60
2260696,60
2260697,60


In [7]:
# getting expected total amount from loan
acc['total_expected'] = acc['installment'] * acc['term']

# round to 2 decimal places
acc[['total_expected']] = acc[['total_expected']].round(2)

acc[['total_expected']]

,total_expected
0,4429.08
1,29530.08
2,25959.60
3,49794.00
4,17394.60
...,...
2260694,32610.00
2260695,30944.40
2260696,51573.60
2260697,33873.60


In [8]:
# getting 70% of total expected payments
acc[['total_expected_70']] = acc[['total_expected']] * .7

# round to 2 decimal places
acc[['total_expected_70']] = acc[['total_expected_70']].round(2)

acc[['total_expected_70']]

,total_expected_70
0,3100.36
1,20671.06
2,18171.72
3,34855.80
4,12176.22
...,...
2260694,22827.00
2260695,21661.08
2260696,36101.52
2260697,23711.52


In [9]:
acc[['funded_amnt', 'total_pymnt']]

,funded_amnt,total_pymnt
0,3600.0,4421.723917
1,24700.0,25679.660000
2,20000.0,22705.924294
3,35000.0,31464.010000
4,10400.0,11740.500000
...,...,...
2260694,24000.0,15760.500000
2260695,24000.0,15008.530000
2260696,40000.0,24903.930000
2260697,24000.0,6755.400000


### Principal 70%

In [10]:
# dataframe of total payment being less than 70% of principal
acc[acc['total_pymnt']<acc['principal_70']]

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,roi,principal_70,total_expected,total_expected_70
13,66624733,NaN,18000.0,18000.0,18000.0,60,19.48,471.70,E,E2,Software Manager,7 years,RENT,150000.0,Not Verified,Dec-2015,Charged Off,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,916xx,CA,9.39,0.0,Jun-2005,665.0,669.0,1.0,NaN,75.0,18.0,1.0,14052.0,40.7,27.0,w,0.00,0.00,9452.74,9452.74,3481.86,4351.98,0.00,1618.90,291.4020,May-2017,471.70,NaN,Nov-2017,584.0,580.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,60.0,36247.0,2.0,2.0,1.0,3.0,2.0,22195.0,71.0,2.0,4.0,1271.0,55.0,34500.0,2.0,1.0,2.0,7.0,2014.0,5128.0,51.3,0.0,0.0,115.0,112.0,1.0,1.0,2.0,9.0,NaN,2.0,NaN,0.0,6.0,13.0,7.0,8.0,6.0,16.0,19.0,13.0,18.0,0.0,0.0,0.0,3.0,100.0,14.3,1.0,0.0,65819.0,36247.0,10300.0,31319.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,-0.474848,12600.0,28302.00,19811.40
25,67849662,NaN,4225.0,4225.0,4225.0,36,14.85,146.16,C,C5,mechanic,5 years,RENT,35000.0,Source Verified,Dec-2015,Charged Off,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,672xx,KS,15.22,2.0,Jul-2011,725.0,729.0,0.0,18.0,NaN,6.0,0.0,1058.0,24.6,6.0,w,0.00,0.00,2558.87,2558.87,1536.90,653.60,0.00,368.37,66.3066,Apr-2017,146.16,NaN,Dec-2017,499.0,0.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,4888.0,0.0,1.0,0.0,0.0,47.0,3830.0,22.0,0.0,0.0,367.0,22.0,4300.0,0.0,0.0,0.0,0.0,815.0,2233.0,14.1,0.0,0.0,47.0,53.0,26.0,26.0,0.0,26.0,NaN,14.0,18.0,0.0,1.0,2.0,3.0,3.0,1.0,5.0,5.0,2.0,

In [11]:
# dataframe of total payment being greater than 70% of principal
acc[acc['total_pymnt']>=acc['principal_70']]

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,roi,principal_70,total_expected,total_expected_70
0,68407277,NaN,3600.0,3600.0,3600.0,36,13.99,123.03,C,C4,leadman,10+ years,MORTGAGE,55000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,190xx,PA,5.91,0.0,Aug-2003,675.0,679.0,1.0,30.0,NaN,7.0,0.0,2765.0,29.7,13.0,w,0.00,0.00,4421.723917,4421.72,3600.00,821.72,0.0,0.0,0.0,Jan-2019,122.67,NaN,Mar-2019,564.0,560.0,0.0,30.0,1.0,Individual,NaN,NaN,NaN,0.0,722.0,144904.0,2.0,2.0,0.0,1.0,21.0,4981.0,36.0,3.0,3.0,722.0,34.0,9300.0,3.0,1.0,4.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,0.0,0.0,0.0,3.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,0.228257,2520.0,4429.08,3100.36
1,68355089,NaN,24700.0,24700.0,24700.0,36,11.99,820.28,C,C1,Engineer,10+ years,MORTGAGE,65000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,small_business,Business,577xx,SD,16.06,1.0,Dec-1999,715.0,719.0,4.0,6.0,NaN,22.0,0.0,21470.0,19.2,38.0,w,0.00,0.00,25679.660000,25679.66,24700.00,979.66,0.0,0.0,0.0,Jun-2016,926.35,NaN,Mar-2019,699.0,695.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,204396.0,1.0,1.0,0.0,1.0,19.0,18005.0,73.0,2.0,3.0,6472.0,29.0,111800.0,0.0,0.0,6.0,4.0,9733.0,57830.0,27.1,0.0,0.0,113.0,192.0,2.0,2.0,4.0,2.0,NaN,0.0,6.0,0.0,5.0,5.0,13.0,17.0,6.0,20.0,27.0,5.0,22.0,0.0,0.0,0.0,2.0,97.

### Total payment 70%

In [12]:
# dataframe of total payment being less than 70% of total payment
acc[acc['total_pymnt']<acc['total_expected_70']]

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,roi,principal_70,total_expected,total_expected_70
3,66310712,NaN,35000.0,35000.0,35000.0,60,14.85,829.90,C,C5,Information Systems Officer,10+ years,MORTGAGE,110000.0,Source Verified,Dec-2015,Current,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,076xx,NJ,17.06,0.0,Sep-2008,785.0,789.0,0.0,NaN,NaN,13.0,0.0,7802.0,11.6,17.0,w,15897.65,15897.65,31464.01,31464.01,19102.35,12361.66,0.00,0.0,0.000,Feb-2019,829.90,Apr-2019,Mar-2019,679.0,675.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,301500.0,1.0,1.0,0.0,1.0,23.0,12609.0,70.0,1.0,1.0,6987.0,45.0,67300.0,0.0,1.0,0.0,2.0,23192.0,54962.0,12.1,0.0,0.0,36.0,87.0,2.0,2.0,1.0,2.0,NaN,NaN,NaN,0.0,4.0,5.0,8.0,10.0,2.0,10.0,13.0,5.0,13.0,0.0,0.0,0.0,1.0,100.0,0.0,0.0,0.0,381215.0,52226.0,62500.0,18000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,-0.101028,24500.0,49794.0,34855.80
4,68476807,NaN,10400.0,10400.0,10400.0,60,22.45,289.91,F,F1,Contract Specialist,3 years,MORTGAGE,104433.0,Source Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,major_purchase,Major purchase,174xx,PA,25.37,1.0,Jun-1998,695.0,699.0,3.0,12.0,NaN,12.0,0.0,21929.0,64.5,35.0,w,0.00,0.00,11740.50,11740.50,10400.00,1340.50,0.00,0.0,0.000,Jul-2016,10128.96,NaN,Mar-2018,704.0,700.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,331730.0,1.0,3.0,0.0,3.0,14.0,73839.0,84.0,4.0,7.0,9702.0,78.0,34000.0,2.0,1.0,3.0,10.0,27644.0,4567.0,77.5,0.0,0.0,128.0,210.0,4.0,4.0,6.0,4.0,12.0,

In [13]:
# dataframe of total payment being greater than 70% of total payment
acc[acc['total_pymnt']>=acc['total_expected_70']]

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,roi,principal_70,total_expected,total_expected_70
0,68407277,NaN,3600.0,3600.0,3600.0,36,13.99,123.03,C,C4,leadman,10+ years,MORTGAGE,55000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,190xx,PA,5.91,0.0,Aug-2003,675.0,679.0,1.0,30.0,NaN,7.0,0.0,2765.0,29.7,13.0,w,0.0,0.0,4421.723917,4421.72,3600.0,821.72,0.0,0.0,0.0,Jan-2019,122.67,NaN,Mar-2019,564.0,560.0,0.0,30.0,1.0,Individual,NaN,NaN,NaN,0.0,722.0,144904.0,2.0,2.0,0.0,1.0,21.0,4981.0,36.0,3.0,3.0,722.0,34.0,9300.0,3.0,1.0,4.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,0.0,0.0,0.0,3.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,0.228257,2520.0,4429.08,3100.36
1,68355089,NaN,24700.0,24700.0,24700.0,36,11.99,820.28,C,C1,Engineer,10+ years,MORTGAGE,65000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,small_business,Business,577xx,SD,16.06,1.0,Dec-1999,715.0,719.0,4.0,6.0,NaN,22.0,0.0,21470.0,19.2,38.0,w,0.0,0.0,25679.660000,25679.66,24700.0,979.66,0.0,0.0,0.0,Jun-2016,926.35,NaN,Mar-2019,699.0,695.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,204396.0,1.0,1.0,0.0,1.0,19.0,18005.0,73.0,2.0,3.0,6472.0,29.0,111800.0,0.0,0.0,6.0,4.0,9733.0,57830.0,27.1,0.0,0.0,113.0,192.0,2.0,2.0,4.0,2.0,NaN,0.0,6.0,0.0,5.0,5.0,13.0,17.0,6.0,20.0,27.0,5.0,22.0,0.0,0.0,0.0,2.0,97.4,7.7,

## We can also look at if the investor can at least make back principal

In [14]:
# dataframe of total payment being less than principal
acc[acc['total_pymnt']<acc['funded_amnt']]

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,roi,principal_70,total_expected,total_expected_70
3,66310712,NaN,35000.0,35000.0,35000.0,60,14.85,829.90,C,C5,Information Systems Officer,10+ years,MORTGAGE,110000.0,Source Verified,Dec-2015,Current,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,076xx,NJ,17.06,0.0,Sep-2008,785.0,789.0,0.0,NaN,NaN,13.0,0.0,7802.0,11.6,17.0,w,15897.65,15897.65,31464.01,31464.01,19102.35,12361.66,0.00,0.0,0.000,Feb-2019,829.90,Apr-2019,Mar-2019,679.0,675.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,301500.0,1.0,1.0,0.0,1.0,23.0,12609.0,70.0,1.0,1.0,6987.0,45.0,67300.0,0.0,1.0,0.0,2.0,23192.0,54962.0,12.1,0.0,0.0,36.0,87.0,2.0,2.0,1.0,2.0,NaN,NaN,NaN,0.0,4.0,5.0,8.0,10.0,2.0,10.0,13.0,5.0,13.0,0.0,0.0,0.0,1.0,100.0,0.0,0.0,0.0,381215.0,52226.0,62500.0,18000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,-0.101028,24500.0,49794.0,34855.80
10,68356421,NaN,22400.0,22400.0,22400.0,60,12.88,508.30,C,C2,Executive Director,6 years,MORTGAGE,95000.0,Not Verified,Dec-2015,Current,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,290xx,SC,22.98,0.0,Apr-1995,710.0,714.0,0.0,54.0,NaN,16.0,0.0,13028.0,32.7,35.0,w,9912.88,9912.88,19275.33,19275.33,12487.12,6788.21,0.00,0.0,0.000,Feb-2019,508.30,Apr-2019,Mar-2019,589.0,585.0,0.0,54.0,1.0,Individual,NaN,NaN,NaN,0.0,0.0,376218.0,1.0,5.0,0.0,3.0,17.0,171328.0,67.0,3.0,6.0,8121.0,54.0,39800.0,1.0,5.0,0.0,9.0,23514.0,12617.0,39.9,0.0,0.0,150.0,248.0,1.0,1.0,5

In [15]:
# dataframe of total payment being greater than principal
acc[acc['total_pymnt']>=acc['funded_amnt']]

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,roi,principal_70,total_expected,total_expected_70
0,68407277,NaN,3600.0,3600.0,3600.0,36,13.99,123.03,C,C4,leadman,10+ years,MORTGAGE,55000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,190xx,PA,5.91,0.0,Aug-2003,675.0,679.0,1.0,30.0,NaN,7.0,0.0,2765.0,29.7,13.0,w,0.0,0.0,4421.723917,4421.72,3600.0,821.72,0.0,0.0,0.0,Jan-2019,122.67,NaN,Mar-2019,564.0,560.0,0.0,30.0,1.0,Individual,NaN,NaN,NaN,0.0,722.0,144904.0,2.0,2.0,0.0,1.0,21.0,4981.0,36.0,3.0,3.0,722.0,34.0,9300.0,3.0,1.0,4.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,0.0,0.0,0.0,3.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,0.228257,2520.0,4429.08,3100.36
1,68355089,NaN,24700.0,24700.0,24700.0,36,11.99,820.28,C,C1,Engineer,10+ years,MORTGAGE,65000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,small_business,Business,577xx,SD,16.06,1.0,Dec-1999,715.0,719.0,4.0,6.0,NaN,22.0,0.0,21470.0,19.2,38.0,w,0.0,0.0,25679.660000,25679.66,24700.0,979.66,0.0,0.0,0.0,Jun-2016,926.35,NaN,Mar-2019,699.0,695.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,204396.0,1.0,1.0,0.0,1.0,19.0,18005.0,73.0,2.0,3.0,6472.0,29.0,111800.0,0.0,0.0,6.0,4.0,9733.0,57830.0,27.1,0.0,0.0,113.0,192.0,2.0,2.0,4.0,2.0,NaN,0.0,6.0,0.0,5.0,5.0,13.0,17.0,6.0,20.0,27.0,5.0,22.0,0.0,0.0,0.0,2.0,97.4,7.7,

- we see that comparing total payment to funded amount (principal) gives us a more even split

# Using Funded Amount as Target

In [16]:
# converting to 1s and 0s

# 1 means UN-able to recoup principal
acc[['investor_expect']] = 1

# 0 means ABLE to recoup principal
acc.loc[acc['total_pymnt']>=acc['funded_amnt'], 'investor_expect'] = 0

acc[['investor_expect']]

,investor_expect
0,0
1,0
2,0
3,1
4,0
...,...
2260694,1
2260695,1
2260696,1
2260697,1


# Using 70% of Expected Payment as Target

In [17]:
# converting to 1s and 0s

# 1 means UN-able to recoup principal
acc[['investor_expect_70total']] = 1

# 0 means ABLE to recoup principal
acc.loc[acc['total_pymnt']>=acc['total_expected_70'], 'investor_expect_70total'] = 0

acc[['investor_expect_70total']]

,investor_expect_70total
0,0
1,0
2,0
3,1
4,0
...,...
2260694,1
2260695,1
2260696,1
2260697,1


# Using 70% of Principal as Target

In [18]:
# converting to 1s and 0s

# 1 means UN-able to recoup principal
acc[['investor_expect_70principal']] = 1

# 0 means ABLE to recoup principal
acc.loc[acc['total_pymnt']>=acc['principal_70'], 'investor_expect_70principal'] = 0

acc[['investor_expect_70principal']]

,investor_expect_70principal
0,0
1,0
2,0
3,1
4,0
...,...
2260694,1
2260695,1
2260696,1
2260697,1


# Data Cleaning and Imputation

## Getting rows that contain the desired target

In [19]:
# keeping only the rows that have loan status of Paid Off, Default, or Charged Off

acc_train = copy.deepcopy(acc[(acc['loan_status'].str.contains('Paid|Default|Charged', na=False))])
acc_train = acc_train.reset_index()
acc_train = acc_train.drop('index', axis=1)
acc_train

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,roi,principal_70,total_expected,total_expected_70,investor_expect,investor_expect_70total,investor_expect_70principal
0,68407277,NaN,3600.0,3600.0,3600.0,36,13.99,123.03,C,C4,leadman,10+ years,MORTGAGE,55000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,190xx,PA,5.91,0.0,Aug-2003,675.0,679.0,1.0,30.0,NaN,7.0,0.0,2765.0,29.7,13.0,w,0.0,0.0,4421.723917,4421.72,3600.00,821.72,0.0,0.0,0.0,Jan-2019,122.67,NaN,Mar-2019,564.0,560.0,0.0,30.0,1.0,Individual,NaN,NaN,NaN,0.0,722.0,144904.0,2.0,2.0,0.0,1.0,21.0,4981.0,36.0,3.0,3.0,722.0,34.0,9300.0,3.0,1.0,4.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,0.0,0.0,0.0,3.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,0.228257,2520.0,4429.08,3100.36,0,0,0
1,68355089,NaN,24700.0,24700.0,24700.0,36,11.99,820.28,C,C1,Engineer,10+ years,MORTGAGE,65000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,small_business,Business,577xx,SD,16.06,1.0,Dec-1999,715.0,719.0,4.0,6.0,NaN,22.0,0.0,21470.0,19.2,38.0,w,0.0,0.0,25679.660000,25679.66,24700.00,979.66,0.0,0.0,0.0,Jun-2016,926.35,NaN,Mar-2019,699.0,695.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,204396.0,1.0,1.0,0.0,1.0,19.0,18005.0,73.0,2.0,3.0,6472.0,29.0,111800.0,0.0,0.0,6.0,4.0,9733.0,57830.0,27.1,0.0,0.0,113.0,192.0,2.0,2.0,4.0,2.0,NaN,0.

In [20]:
# this is the benchmark

acc_train['roi'] = (acc_train['total_pymnt'] - acc_train['funded_amnt'])/acc_train['funded_amnt']

acc_train['roi'].mean()

0.033766986328038553

In [21]:
acc_train['pnl'] = acc_train['total_pymnt'] - acc_train['funded_amnt']

acc_train['pnl'].mean()

411.51834540676407

In [21]:
# create target label

target_train = acc_train[['investor_expect',
                          'investor_expect_70total',
                          'investor_expect_70principal',
                          'pnl']]

target_train

,investor_expect,investor_expect_70total,investor_expect_70principal
0,0,0,0
1,0,0,0
2,0,0,0
3,0,0,0
4,0,0,0
...,...,...,...
1348094,0,0,0
1348095,0,0,0
1348096,1,1,1
1348097,0,0,0


In [22]:
target_train[['investor_expect']].value_counts()

investor_expect
0                  1097706
1                   250393
dtype: int64

In [23]:
target_train[['investor_expect_70total']].value_counts()

investor_expect_70total
0                          1097706
1                           250393
dtype: int64

In [24]:
target_train[['investor_expect_70principal']].value_counts()

investor_expect_70principal
0                              1097706
1                               250393
dtype: int64

In [25]:
acc_train = acc_train.drop(['investor_expect',
                            'investor_expect_70total',
                            'investor_expect_70principal'], axis=1)

# Splitting the dataset into train and prediction

### Funded Amount

In [26]:
# getting indices of half of the samples

target_funded_index_0 = sorted(list(target_train\
                                    [target_train\
                                     ['investor_expect']==0].sample(frac=0.5,
                                                                    random_state=0).index))

target_funded_index_1 = sorted(list(target_train\
                                    [target_train\
                                     ['investor_expect']==1].sample(frac=0.5,
                                                                    random_state=0).index))

target_funded_index_0and1 = target_funded_index_0 + target_funded_index_1


# getting the other half of the indices

target_funded_pred_index = sorted(list(set(list(range(0,
                                                      target_train['investor_expect'].count()))) -\
                                       set(target_funded_index_0and1)))

In [28]:
len(target_funded_index_0and1)

674049

In [29]:
len(target_funded_pred_index)

674050

In [30]:
# separating the dataframe into 2 dataframes: 1 for use/train and 1 for prediction comparison

acc_3_train_funded = acc_train.iloc[target_funded_index_0and1]
acc_3_train_funded = acc_3_train_funded.reset_index()
acc_3_train_funded = acc_3_train_funded.drop('index', axis=1)

target_3_train_funded = target_train.iloc[target_funded_index_0and1]
target_3_train_funded = target_3_train_funded.reset_index()
target_3_train_funded = target_3_train_funded.drop('index', axis=1)


acc_3_pred_funded = acc_train.iloc[target_funded_pred_index]
acc_3_pred_funded = acc_3_pred_funded.reset_index()
acc_3_pred_funded = acc_3_pred_funded.drop('index', axis=1)

target_3_pred_funded = target_train.iloc[target_funded_pred_index]
target_3_pred_funded = target_3_pred_funded.reset_index()
target_3_pred_funded = target_3_pred_funded.drop('index', axis=1)


acc_3_train_funded

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,roi,principal_70,total_expected,total_expected_70
0,68407277,NaN,3600.0,3600.0,3600.0,36,13.99,123.03,C,C4,leadman,10+ years,MORTGAGE,55000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,190xx,PA,5.91,0.0,Aug-2003,675.0,679.0,1.0,30.0,NaN,7.0,0.0,2765.0,29.7,13.0,w,0.0,0.0,4421.723917,4421.72,3600.00,821.72,0.0,0.00,0.0000,Jan-2019,122.67,NaN,Mar-2019,564.0,560.0,0.0,30.0,1.0,Individual,NaN,NaN,NaN,0.0,722.0,144904.0,2.0,2.0,0.0,1.0,21.0,4981.0,36.0,3.0,3.0,722.0,34.0,9300.0,3.0,1.0,4.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,0.0,0.0,0.0,3.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,0.228257,2520.0,4429.08,3100.36
1,68476668,NaN,20000.0,20000.0,20000.0,36,9.17,637.58,B,B2,Vice President of Recruiting Operations,10+ years,MORTGAGE,180000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,550xx,MN,14.67,0.0,Jun-1990,680.0,684.0,0.0,49.0,NaN,12.0,0.0,87329.0,84.5,27.0,f,0.0,0.0,21393.800000,21393.80,20000.00,1393.80,0.0,0.00,0.0000,Nov-2016,15681.05,NaN,Mar-2019,654.0,650.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,360358.0,0.0,2.0,0.0,2.0,18.0,29433.0,63.0,2.0,3.0,13048.0,74.0,94200.0,1.0,0.0,1.0,6.0,30030.0,0.0,102.9,0.0,0.0,142.0,306.0,10.0,10.0,4.0,12.0,NaN,10.0,NaN,0.0,4.0,6

In [31]:
target_3_train_funded

,investor_expect,investor_expect_70total,investor_expect_70principal
0,0,0,0
1,0,0,0
2,0,0,0
3,0,0,0
4,0,0,0
...,...,...,...
674044,1,1,1
674045,1,1,1
674046,1,1,1
674047,1,1,1


### 70% of Expected Payment

In [32]:
# getting indices of half of the samples

target_70total_index_0 = sorted(list(target_train\
                                    [target_train\
                                     ['investor_expect_70total']==0].sample(frac=0.5,
                                                                            random_state=0).index))

target_70total_index_1 = sorted(list(target_train\
                                    [target_train\
                                     ['investor_expect_70total']==1].sample(frac=0.5,
                                                                            random_state=0).index))

target_70total_index_0and1 = target_70total_index_0 + target_70total_index_1


# getting the other half of the indices

target_70total_pred_index = sorted(list(set(list(range(0,
                                                      target_train\
                                                       ['investor_expect_70total'].count()))) -\
                                        set(target_70total_index_0and1)))

In [33]:
print(len(target_70total_index_0and1))
len(target_70total_pred_index)

674049


674050

In [34]:
# separating the dataframe into 2 dataframes: 1 for use/train and 1 for prediction comparison

acc_3_train_70total = acc_train.iloc[target_70total_index_0and1]
acc_3_train_70total = acc_3_train_70total.reset_index()
acc_3_train_70total = acc_3_train_70total.drop('index', axis=1)

target_3_train_70total = target_train.iloc[target_70total_index_0and1]
target_3_train_70total = target_3_train_70total.reset_index()
target_3_train_70total = target_3_train_70total.drop('index', axis=1)


acc_3_pred_70total = acc_train.iloc[target_70total_pred_index]
acc_3_pred_70total = acc_3_pred_70total.reset_index()
acc_3_pred_70total = acc_3_pred_70total.drop('index', axis=1)

target_3_pred_70total = target_train.iloc[target_70total_pred_index]
target_3_pred_70total = target_3_pred_70total.reset_index()
target_3_pred_70total = target_3_pred_70total.drop('index', axis=1)


acc_3_train_70total

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,roi,principal_70,total_expected,total_expected_70
0,68407277,NaN,3600.0,3600.0,3600.0,36,13.99,123.03,C,C4,leadman,10+ years,MORTGAGE,55000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,190xx,PA,5.91,0.0,Aug-2003,675.0,679.0,1.0,30.0,NaN,7.0,0.0,2765.0,29.7,13.0,w,0.0,0.0,4421.723917,4421.72,3600.00,821.72,0.0,0.00,0.0000,Jan-2019,122.67,NaN,Mar-2019,564.0,560.0,0.0,30.0,1.0,Individual,NaN,NaN,NaN,0.0,722.0,144904.0,2.0,2.0,0.0,1.0,21.0,4981.0,36.0,3.0,3.0,722.0,34.0,9300.0,3.0,1.0,4.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,0.0,0.0,0.0,3.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,0.228257,2520.0,4429.08,3100.36
1,68476668,NaN,20000.0,20000.0,20000.0,36,9.17,637.58,B,B2,Vice President of Recruiting Operations,10+ years,MORTGAGE,180000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,550xx,MN,14.67,0.0,Jun-1990,680.0,684.0,0.0,49.0,NaN,12.0,0.0,87329.0,84.5,27.0,f,0.0,0.0,21393.800000,21393.80,20000.00,1393.80,0.0,0.00,0.0000,Nov-2016,15681.05,NaN,Mar-2019,654.0,650.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,360358.0,0.0,2.0,0.0,2.0,18.0,29433.0,63.0,2.0,3.0,13048.0,74.0,94200.0,1.0,0.0,1.0,6.0,30030.0,0.0,102.9,0.0,0.0,142.0,306.0,10.0,10.0,4.0,12.0,NaN,10.0,NaN,0.0,4.0,6

In [35]:
target_3_train_70total

,investor_expect,investor_expect_70total,investor_expect_70principal
0,0,0,0
1,0,0,0
2,0,0,0
3,0,0,0
4,0,0,0
...,...,...,...
674044,1,1,1
674045,1,1,1
674046,1,1,1
674047,1,1,1


### 70% of Principal

In [36]:
# getting indices of half of the samples

target_70principal_index_0 = sorted(list(target_train\
                                    [target_train\
                                     ['investor_expect_70principal']==0].sample(frac=0.5,
                                                                    random_state=0).index))

target_70principal_index_1 = sorted(list(target_train\
                                    [target_train\
                                     ['investor_expect_70principal']==1].sample(frac=0.5,
                                                                    random_state=0).index))

target_70principal_index_0and1 = target_70principal_index_0 + target_70principal_index_1


# getting the other half of the indices

target_70principal_pred_index = sorted(list(set\
                                            (list(range(0,
                                                        target_train\
                                                        ['investor_expect_70principal'].count()))) -\
                                            set(target_70principal_index_0and1)))

In [37]:
print(len(target_70principal_index_0and1))
len(target_70principal_pred_index)

674049


674050

In [38]:
# separating the dataframe into 2 dataframes: 1 for use/train and 1 for prediction comparison

acc_3_train_70principal = acc_train.iloc[target_70principal_index_0and1]
acc_3_train_70principal = acc_3_train_70principal.reset_index()
acc_3_train_70principal = acc_3_train_70principal.drop('index', axis=1)

target_3_train_70principal = target_train.iloc[target_70principal_index_0and1]
target_3_train_70principal = target_3_train_70principal.reset_index()
target_3_train_70principal = target_3_train_70principal.drop('index', axis=1)


acc_3_pred_70principal = acc_train.iloc[target_70principal_pred_index]
acc_3_pred_70principal = acc_3_pred_70principal.reset_index()
acc_3_pred_70principal = acc_3_pred_70principal.drop('index', axis=1)

target_3_pred_70principal = target_train.iloc[target_70principal_pred_index]
target_3_pred_70principal = target_3_pred_70principal.reset_index()
target_3_pred_70principal = target_3_pred_70principal.drop('index', axis=1)


acc_3_train_70principal

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,roi,principal_70,total_expected,total_expected_70
0,68407277,NaN,3600.0,3600.0,3600.0,36,13.99,123.03,C,C4,leadman,10+ years,MORTGAGE,55000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,190xx,PA,5.91,0.0,Aug-2003,675.0,679.0,1.0,30.0,NaN,7.0,0.0,2765.0,29.7,13.0,w,0.0,0.0,4421.723917,4421.72,3600.00,821.72,0.0,0.00,0.0000,Jan-2019,122.67,NaN,Mar-2019,564.0,560.0,0.0,30.0,1.0,Individual,NaN,NaN,NaN,0.0,722.0,144904.0,2.0,2.0,0.0,1.0,21.0,4981.0,36.0,3.0,3.0,722.0,34.0,9300.0,3.0,1.0,4.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,0.0,0.0,0.0,3.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,0.228257,2520.0,4429.08,3100.36
1,68476668,NaN,20000.0,20000.0,20000.0,36,9.17,637.58,B,B2,Vice President of Recruiting Operations,10+ years,MORTGAGE,180000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,550xx,MN,14.67,0.0,Jun-1990,680.0,684.0,0.0,49.0,NaN,12.0,0.0,87329.0,84.5,27.0,f,0.0,0.0,21393.800000,21393.80,20000.00,1393.80,0.0,0.00,0.0000,Nov-2016,15681.05,NaN,Mar-2019,654.0,650.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,360358.0,0.0,2.0,0.0,2.0,18.0,29433.0,63.0,2.0,3.0,13048.0,74.0,94200.0,1.0,0.0,1.0,6.0,30030.0,0.0,102.9,0.0,0.0,142.0,306.0,10.0,10.0,4.0,12.0,NaN,10.0,NaN,0.0,4.0,6

In [39]:
target_3_train_70principal

,investor_expect,investor_expect_70total,investor_expect_70principal
0,0,0,0
1,0,0,0
2,0,0,0
3,0,0,0
4,0,0,0
...,...,...,...
674044,1,1,1
674045,1,1,1
674046,1,1,1
674047,1,1,1


# SHOULD WE BE DROPPING ROWS RN?

In [40]:
# drop unneeded columns

acc_clean_funded = acc_3_train_funded.drop(['collection_recovery_fee', 'desc', 'id', 'member_id',
                                          'next_pymnt_d', 'out_prncp', 'out_prncp_inv','pymnt_plan',
                                          'recoveries', 'title', 'total_pymnt_inv',
                                          'total_rec_int', 'total_rec_late_fee', 'total_rec_prncp',
                                          'url', 'hardship_flag', 'hardship_type', 'hardship_reason',
                                          'hardship_status', 'deferral_term', 'hardship_amount',
                                          'hardship_start_date', 'hardship_end_date',
                                          'payment_plan_start_date', 'hardship_length',
                                          'hardship_dpd', 'hardship_loan_status',
                                          'orig_projected_additional_accrued_interest',
                                          'hardship_payoff_balance_amount',
                                          'hardship_last_payment_amount', 'disbursement_method',
                                          'debt_settlement_flag', 'debt_settlement_flag_date',
                                          'settlement_status', 'settlement_date', 'settlement_amount',
                                          'settlement_percentage', 'settlement_term', 'last_pymnt_d',
                                          'last_pymnt_amnt', 'last_fico_range_high',
                                          'last_fico_range_low', 'inq_last_12m', 'inq_last_6mths',
                                          'open_rv_24m', 'open_rv_12m', 'open_il_24m', 'open_il_12m',
                                          'open_acc_6m', 'num_tl_op_past_12m', 'num_tl_90g_dpd_24m',
                                          'num_tl_30dpd', 'num_tl_120dpd_2m', 'num_accts_ever_120_pd',
                                          'sec_app_chargeoff_within_12_mths',
                                          'sec_app_collections_12_mths_ex_med', 'zip_code',
                                          'addr_state'], axis=1)

acc_clean_funded

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,purpose,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,total_pymnt,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_act_il,mths_since_rcnt_il,total_bal_il,il_util,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_mths_since_last_major_derog,roi,principal_70,total_expected,total_expected_70
0,3600.0,3600.0,3600.0,36,13.99,123.03,C,C4,leadman,10+ years,MORTGAGE,55000.0,Not Verified,Dec-2015,Fully Paid,debt_consolidation,5.91,0.0,Aug-2003,675.0,679.0,30.0,NaN,7.0,0.0,2765.0,29.7,13.0,w,4421.723917,Mar-2019,0.0,30.0,1.0,Individual,NaN,NaN,NaN,0.0,722.0,144904.0,2.0,21.0,4981.0,36.0,722.0,34.0,9300.0,3.0,1.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.228257,2520.0,4429.08,3100.36
1,20000.0,20000.0,20000.0,36,9.17,637.58,B,B2,Vice President of Recruiting Operations,10+ years,MORTGAGE,180000.0,Not Verified,Dec-2015,Fully Paid,debt_consolidation,14.67,0.0,Jun-1990,680.0,684.0,49.0,NaN,12.0,0.0,87329.0,84.5,27.0,f,21393.800000,Mar-2019,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,360358.0,2.0,18.0,29433.0,63.0,13048.0,74.0,94200.0,1.0,0.0,6.0,30030.0,0.0,102.9,0.0,0.0,142.0,306.0,10.0,10.0,4.0,12.0,NaN,10.0,NaN,4.0,6.0,4.0,5.0,7.0,9.0,16.0,6.0,12.0,96.3,100.0,0.0,0.0,388852.0,116762.0,31500.0,46452.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.069690,14000.0,22952.88,16067.02
2,20000.0,20000.0,20000.0,36,8.49,631.26,B,B1,road driver,10+ years,MORTGAGE,85000.0,Not Verified,Dec-2015,Fully Paid,major_purchase,17.61,1.0,Feb-1999,705.0,709.0,3.0,NaN,8.0,0.0,826.0,5.7,15.0,w,21538.508977,Mar-2019,0.0,3.0,1.0,Individual,NaN,NaN,NaN,0.0,0.0,141601.0,3.0,13.0,27111.0,75.0,640.0,55.0,14500.0,1.0,0.0,4.0,17700.0,13674.0,5.7,0.0,0.0,149.0,55.0,32.0,13.0,3.0,32.0,NaN,8.0,NaN,2.0,2.0,3.0,3.0,9.0,3.0,3.0,2.0,8.0,93.3,0.0,0.0,0.0,193390.0,27937.0,14500.0,36144.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.076925,14000.0,22725.36,15907.75
3,10000.0,10000.0,10000.0,36,6.49,306.45,A,A2,SERVICE MANAGER,6 years,RENT,85000.0,Not Verified,Dec-2015,Fully Paid,credit_card,13.07,0.0,Apr-2002,685.0,689.0,NaN,106.0,14.0,1.0,10464.0,34.5,23.0,w,10998.971575,Mar-2019,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,8341.0,27957.0,1.0,35.0,17493.0,57.0,2524.0,46.0,30300.0,2.0,0.0,7.0,1997.0,8182.0,50.1,0.0,0.0,164.0,129.0,1.0,1.0,1.0,4.0,NaN,1.0,NaN,6.0,9.0,7.0,10.0,3.0,13.0,19.0,9.0,14.0,95.7,28.6,1.0,0.0,61099.0,27957.0,16400.0,30799.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.099897,7000.0,11032.20,7722.54
4,8000.0,8000.0,8000.0,36,11.48,263.74,B,B5,Vendor liaison,10+ years,MORTGAGE,42000.0,Not Verified,Dec-2015,Fully Paid,credit_card,34.80,0.0,Nov-1994,700.0,704.0,75.0,NaN,8.0,0.0,7034.0,39.1,18.0,w,8939.580503,Nov-2018,0.0

In [41]:
# drop unneeded columns

acc_clean_70total = acc_3_train_70total.drop(['collection_recovery_fee', 'desc', 'id', 'member_id',
                                          'next_pymnt_d', 'out_prncp', 'out_prncp_inv','pymnt_plan',
                                          'recoveries', 'title', 'total_pymnt_inv',
                                          'total_rec_int', 'total_rec_late_fee', 'total_rec_prncp',
                                          'url', 'hardship_flag', 'hardship_type', 'hardship_reason',
                                          'hardship_status', 'deferral_term', 'hardship_amount',
                                          'hardship_start_date', 'hardship_end_date',
                                          'payment_plan_start_date', 'hardship_length',
                                          'hardship_dpd', 'hardship_loan_status',
                                          'orig_projected_additional_accrued_interest',
                                          'hardship_payoff_balance_amount',
                                          'hardship_last_payment_amount', 'disbursement_method',
                                          'debt_settlement_flag', 'debt_settlement_flag_date',
                                          'settlement_status', 'settlement_date', 'settlement_amount',
                                          'settlement_percentage', 'settlement_term', 'last_pymnt_d',
                                          'last_pymnt_amnt', 'last_fico_range_high',
                                          'last_fico_range_low', 'inq_last_12m', 'inq_last_6mths',
                                          'open_rv_24m', 'open_rv_12m', 'open_il_24m', 'open_il_12m',
                                          'open_acc_6m', 'num_tl_op_past_12m', 'num_tl_90g_dpd_24m',
                                          'num_tl_30dpd', 'num_tl_120dpd_2m', 'num_accts_ever_120_pd',
                                          'sec_app_chargeoff_within_12_mths',
                                          'sec_app_collections_12_mths_ex_med', 'zip_code',
                                          'addr_state'], axis=1)

acc_clean_funded

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,purpose,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,total_pymnt,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_act_il,mths_since_rcnt_il,total_bal_il,il_util,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_mths_since_last_major_derog,roi,principal_70,total_expected,total_expected_70
0,3600.0,3600.0,3600.0,36,13.99,123.03,C,C4,leadman,10+ years,MORTGAGE,55000.0,Not Verified,Dec-2015,Fully Paid,debt_consolidation,5.91,0.0,Aug-2003,675.0,679.0,30.0,NaN,7.0,0.0,2765.0,29.7,13.0,w,4421.723917,Mar-2019,0.0,30.0,1.0,Individual,NaN,NaN,NaN,0.0,722.0,144904.0,2.0,21.0,4981.0,36.0,722.0,34.0,9300.0,3.0,1.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.228257,2520.0,4429.08,3100.36
1,20000.0,20000.0,20000.0,36,9.17,637.58,B,B2,Vice President of Recruiting Operations,10+ years,MORTGAGE,180000.0,Not Verified,Dec-2015,Fully Paid,debt_consolidation,14.67,0.0,Jun-1990,680.0,684.0,49.0,NaN,12.0,0.0,87329.0,84.5,27.0,f,21393.800000,Mar-2019,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,360358.0,2.0,18.0,29433.0,63.0,13048.0,74.0,94200.0,1.0,0.0,6.0,30030.0,0.0,102.9,0.0,0.0,142.0,306.0,10.0,10.0,4.0,12.0,NaN,10.0,NaN,4.0,6.0,4.0,5.0,7.0,9.0,16.0,6.0,12.0,96.3,100.0,0.0,0.0,388852.0,116762.0,31500.0,46452.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.069690,14000.0,22952.88,16067.02
2,20000.0,20000.0,20000.0,36,8.49,631.26,B,B1,road driver,10+ years,MORTGAGE,85000.0,Not Verified,Dec-2015,Fully Paid,major_purchase,17.61,1.0,Feb-1999,705.0,709.0,3.0,NaN,8.0,0.0,826.0,5.7,15.0,w,21538.508977,Mar-2019,0.0,3.0,1.0,Individual,NaN,NaN,NaN,0.0,0.0,141601.0,3.0,13.0,27111.0,75.0,640.0,55.0,14500.0,1.0,0.0,4.0,17700.0,13674.0,5.7,0.0,0.0,149.0,55.0,32.0,13.0,3.0,32.0,NaN,8.0,NaN,2.0,2.0,3.0,3.0,9.0,3.0,3.0,2.0,8.0,93.3,0.0,0.0,0.0,193390.0,27937.0,14500.0,36144.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.076925,14000.0,22725.36,15907.75
3,10000.0,10000.0,10000.0,36,6.49,306.45,A,A2,SERVICE MANAGER,6 years,RENT,85000.0,Not Verified,Dec-2015,Fully Paid,credit_card,13.07,0.0,Apr-2002,685.0,689.0,NaN,106.0,14.0,1.0,10464.0,34.5,23.0,w,10998.971575,Mar-2019,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,8341.0,27957.0,1.0,35.0,17493.0,57.0,2524.0,46.0,30300.0,2.0,0.0,7.0,1997.0,8182.0,50.1,0.0,0.0,164.0,129.0,1.0,1.0,1.0,4.0,NaN,1.0,NaN,6.0,9.0,7.0,10.0,3.0,13.0,19.0,9.0,14.0,95.7,28.6,1.0,0.0,61099.0,27957.0,16400.0,30799.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.099897,7000.0,11032.20,7722.54
4,8000.0,8000.0,8000.0,36,11.48,263.74,B,B5,Vendor liaison,10+ years,MORTGAGE,42000.0,Not Verified,Dec-2015,Fully Paid,credit_card,34.80,0.0,Nov-1994,700.0,704.0,75.0,NaN,8.0,0.0,7034.0,39.1,18.0,w,8939.580503,Nov-2018,0.0

In [42]:
# drop unneeded columns

acc_clean_70prcp = acc_3_train_70principal.drop(['collection_recovery_fee', 'desc', 'id', 'member_id',
                                          'next_pymnt_d', 'out_prncp', 'out_prncp_inv','pymnt_plan',
                                          'recoveries', 'title', 'total_pymnt_inv',
                                          'total_rec_int', 'total_rec_late_fee', 'total_rec_prncp',
                                          'url', 'hardship_flag', 'hardship_type', 'hardship_reason',
                                          'hardship_status', 'deferral_term', 'hardship_amount',
                                          'hardship_start_date', 'hardship_end_date',
                                          'payment_plan_start_date', 'hardship_length',
                                          'hardship_dpd', 'hardship_loan_status',
                                          'orig_projected_additional_accrued_interest',
                                          'hardship_payoff_balance_amount',
                                          'hardship_last_payment_amount', 'disbursement_method',
                                          'debt_settlement_flag', 'debt_settlement_flag_date',
                                          'settlement_status', 'settlement_date', 'settlement_amount',
                                          'settlement_percentage', 'settlement_term', 'last_pymnt_d',
                                          'last_pymnt_amnt', 'last_fico_range_high',
                                          'last_fico_range_low', 'inq_last_12m', 'inq_last_6mths',
                                          'open_rv_24m', 'open_rv_12m', 'open_il_24m', 'open_il_12m',
                                          'open_acc_6m', 'num_tl_op_past_12m', 'num_tl_90g_dpd_24m',
                                          'num_tl_30dpd', 'num_tl_120dpd_2m', 'num_accts_ever_120_pd',
                                          'sec_app_chargeoff_within_12_mths',
                                          'sec_app_collections_12_mths_ex_med', 'zip_code',
                                          'addr_state'], axis=1)

acc_clean_funded

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,purpose,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,total_pymnt,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_act_il,mths_since_rcnt_il,total_bal_il,il_util,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_mths_since_last_major_derog,roi,principal_70,total_expected,total_expected_70
0,3600.0,3600.0,3600.0,36,13.99,123.03,C,C4,leadman,10+ years,MORTGAGE,55000.0,Not Verified,Dec-2015,Fully Paid,debt_consolidation,5.91,0.0,Aug-2003,675.0,679.0,30.0,NaN,7.0,0.0,2765.0,29.7,13.0,w,4421.723917,Mar-2019,0.0,30.0,1.0,Individual,NaN,NaN,NaN,0.0,722.0,144904.0,2.0,21.0,4981.0,36.0,722.0,34.0,9300.0,3.0,1.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.228257,2520.0,4429.08,3100.36
1,20000.0,20000.0,20000.0,36,9.17,637.58,B,B2,Vice President of Recruiting Operations,10+ years,MORTGAGE,180000.0,Not Verified,Dec-2015,Fully Paid,debt_consolidation,14.67,0.0,Jun-1990,680.0,684.0,49.0,NaN,12.0,0.0,87329.0,84.5,27.0,f,21393.800000,Mar-2019,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,360358.0,2.0,18.0,29433.0,63.0,13048.0,74.0,94200.0,1.0,0.0,6.0,30030.0,0.0,102.9,0.0,0.0,142.0,306.0,10.0,10.0,4.0,12.0,NaN,10.0,NaN,4.0,6.0,4.0,5.0,7.0,9.0,16.0,6.0,12.0,96.3,100.0,0.0,0.0,388852.0,116762.0,31500.0,46452.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.069690,14000.0,22952.88,16067.02
2,20000.0,20000.0,20000.0,36,8.49,631.26,B,B1,road driver,10+ years,MORTGAGE,85000.0,Not Verified,Dec-2015,Fully Paid,major_purchase,17.61,1.0,Feb-1999,705.0,709.0,3.0,NaN,8.0,0.0,826.0,5.7,15.0,w,21538.508977,Mar-2019,0.0,3.0,1.0,Individual,NaN,NaN,NaN,0.0,0.0,141601.0,3.0,13.0,27111.0,75.0,640.0,55.0,14500.0,1.0,0.0,4.0,17700.0,13674.0,5.7,0.0,0.0,149.0,55.0,32.0,13.0,3.0,32.0,NaN,8.0,NaN,2.0,2.0,3.0,3.0,9.0,3.0,3.0,2.0,8.0,93.3,0.0,0.0,0.0,193390.0,27937.0,14500.0,36144.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.076925,14000.0,22725.36,15907.75
3,10000.0,10000.0,10000.0,36,6.49,306.45,A,A2,SERVICE MANAGER,6 years,RENT,85000.0,Not Verified,Dec-2015,Fully Paid,credit_card,13.07,0.0,Apr-2002,685.0,689.0,NaN,106.0,14.0,1.0,10464.0,34.5,23.0,w,10998.971575,Mar-2019,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,8341.0,27957.0,1.0,35.0,17493.0,57.0,2524.0,46.0,30300.0,2.0,0.0,7.0,1997.0,8182.0,50.1,0.0,0.0,164.0,129.0,1.0,1.0,1.0,4.0,NaN,1.0,NaN,6.0,9.0,7.0,10.0,3.0,13.0,19.0,9.0,14.0,95.7,28.6,1.0,0.0,61099.0,27957.0,16400.0,30799.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.099897,7000.0,11032.20,7722.54
4,8000.0,8000.0,8000.0,36,11.48,263.74,B,B5,Vendor liaison,10+ years,MORTGAGE,42000.0,Not Verified,Dec-2015,Fully Paid,credit_card,34.80,0.0,Nov-1994,700.0,704.0,75.0,NaN,8.0,0.0,7034.0,39.1,18.0,w,8939.580503,Nov-2018,0.0

In [46]:
datasets_3 = [acc_clean_funded, acc_clean_70total, acc_clean_70prcp]
              
targets_3 = [target_3_train_funded, target_3_train_70total, target_3_train_70principal]

## For Loop to convert all 3 datasets

In [47]:
for dataset in datasets_3:
    
    # if employment length is 10+ years, insert 10 years
    dataset['emp_length'] = np.where(dataset['emp_length']=='10+ years',
                                       '10 years', dataset['emp_length'])

    # if employment length is 1 year , insert 1 years
    dataset['emp_length'] = np.where(dataset['emp_length']=='1 year',
                                       '1 years', dataset['emp_length'])

    # if employment length is <1, insert 0
    dataset['emp_length'] = np.where(dataset['emp_length']=='< 1 year',
                                       '0 years', dataset['emp_length'])

    # if employment length is null, insert -1
    dataset['emp_length'] = np.where(dataset['emp_length'].isnull(),
                                       '-1 years', dataset['emp_length'])
    # removing ' years' from the column
    dataset['emp_length'] = dataset['emp_length'].str.replace(' years', '', regex=False)

    # Convert to numeric type
    dataset['emp_length'] = pd.to_numeric(dataset['emp_length'])
    
    # Check if columns with dtype object should be ordinal
    # Get columns with string values
    cols_w_strings = dataset.loc[:, dataset.dtypes == object].columns
    
    # Home Ownership
    # Assigning values to home_ownership status
    dataset.loc[dataset['home_ownership']=='OWN', 'home_ownership'] = 3
    dataset.loc[dataset['home_ownership']=='MORTGAGE', 'home_ownership'] = 2
    dataset.loc[dataset['home_ownership']=='ANY', 'home_ownership'] = 2
    dataset.loc[dataset['home_ownership']=='OTHER', 'home_ownership'] = 1
    dataset.loc[dataset['home_ownership']=='RENT', 'home_ownership'] = 0
    dataset.loc[dataset['home_ownership']=='NONE', 'home_ownership'] = 0
    dataset['home_ownership'] = dataset['home_ownership'].astype(int)
    
    # Grade
    # Grouping E, F, and G into F
    dataset['grouped_grade_NO_EG'] = dataset['grade']
    dataset['grouped_grade_NO_EG'] = dataset['grouped_grade_NO_EG'].str.replace('E', 'F',
                                                                                    regex=False)
    dataset['grouped_grade_NO_EG'] = dataset['grouped_grade_NO_EG'].str.replace('G', 'F',
                                                                                    regex=False)
    
    # replacing letter grades with numbers
    # get sorted list of grades
    grade_list = sorted(list(dataset['grade'].unique()))
    i = 1
    for grades in grade_list:
        dataset.loc[dataset['grade']==grades, 'grade'] = i
        i+=1
    dataset['grade'] = dataset['grade'].astype(int)
    dataset['grade'].unique()
    
    # replacing letter grades with numbers for grouped grades
    # get sorted list of grades
    grade_list = sorted(list(dataset['grouped_grade_NO_EG'].unique()))
    i = 1
    for grades in grade_list:
        dataset.loc[dataset['grouped_grade_NO_EG']==grades, 'grouped_grade_NO_EG'] = i
        i+=1
    dataset['grouped_grade_NO_EG'] = dataset['grouped_grade_NO_EG'].astype(int)
    dataset['grouped_grade_NO_EG'].unique()
    
    # Subgrade
    # grouping subgrades 1, 2 vs 3, 4, 5
    dataset['grouped_subgrade_12_345'] = dataset['sub_grade']
    dataset['grouped_subgrade_12_345'] = dataset\
    ['grouped_subgrade_12_345'].str.replace('2','1',regex=False)
    dataset['grouped_subgrade_12_345'] = dataset\
    ['grouped_subgrade_12_345'].str.replace('3','2', regex=False)
    dataset['grouped_subgrade_12_345'] = dataset\
    ['grouped_subgrade_12_345'].str.replace('4','2', regex=False)
    dataset['grouped_subgrade_12_345'] = dataset\
    ['grouped_subgrade_12_345'].str.replace('5','2', regex=False)

    # grouping subgrades 1, 2, 3 vs 4, 5
    dataset['grouped_subgrade_123_45'] = dataset['sub_grade']
    dataset['grouped_subgrade_123_45'] = dataset\
    ['grouped_subgrade_123_45'].str.replace('2','1', regex=False)
    dataset['grouped_subgrade_123_45'] = dataset\
    ['grouped_subgrade_123_45'].str.replace('3','1', regex=False)
    dataset['grouped_subgrade_123_45'] = dataset\
    ['grouped_subgrade_123_45'].str.replace('4','2', regex=False)
    dataset['grouped_subgrade_123_45'] = dataset\
    ['grouped_subgrade_123_45'].str.replace('5','2', regex=False)
    dataset['grouped_subgrade_123_45'].value_counts()

    # grouping Subgrades
    dataset['grouped_subgrade'] = dataset['sub_grade']
    dataset['grouped_subgrade'] = dataset['grouped_subgrade'].str.replace('E','F', regex=False)
    dataset['grouped_subgrade'] = dataset['grouped_subgrade'].str.replace('G','F', regex=False)
    
    # replacing subgrades with numbers
    # get sorted list of subgrades
    subgrade_list = sorted(list(dataset['sub_grade'].unique()))
    i = 1
    for subgrades in subgrade_list:
        dataset.loc[dataset['sub_grade']==subgrades, 'sub_grade'] = i
        i+=1
    dataset['sub_grade'] = dataset['sub_grade'].astype(int)
    
    # replacing subgrades with numbers
    # get sorted list of subgrades
    subgrade_list = sorted(list(dataset['grouped_subgrade_12_345'].unique()))
    i = 1
    for subgrades in subgrade_list:
        dataset.loc[dataset['grouped_subgrade_12_345']==subgrades, 'grouped_subgrade_12_345'] = i
        i+=1
    dataset['grouped_subgrade_12_345'] = dataset['grouped_subgrade_12_345'].astype(int)
    
    # replacing subgrades with numbers
    # get sorted list of subgrades
    subgrade_list = sorted(list(dataset['grouped_subgrade_123_45'].unique()))
    i = 1
    for subgrades in subgrade_list:
        dataset.loc[dataset['grouped_subgrade_123_45']==subgrades, 'grouped_subgrade_123_45'] = i
        i+=1
    dataset['grouped_subgrade_123_45'] = dataset['grouped_subgrade_123_45'].astype(int)
    
    # replacing subgrades with numbers
    # get sorted list of subgrades
    subgrade_list = sorted(list(dataset['grouped_subgrade'].unique()))
    i = 1
    for subgrades in subgrade_list:
        dataset.loc[dataset['grouped_subgrade']==subgrades, 'grouped_subgrade'] = i
        i+=1
    dataset['grouped_subgrade'] = dataset['grouped_subgrade'].astype(int)
    
    # Earliest Credit Line
    # converting the column to datetime format
    dataset['earliest_cr_line_dt'] = pd.DatetimeIndex(dataset['earliest_cr_line'])
    
    # creating a new column that will show the number of months that credit has existed for customer
    dataset['duration_of_credit_months'] = (pd.DatetimeIndex(dataset['issue_d']).year -\
                                              pd.DatetimeIndex(dataset['earliest_cr_line']).year)\
    *12 + (pd.DatetimeIndex(dataset['issue_d']).month -\
           pd.DatetimeIndex(dataset['earliest_cr_line']).month)

    # dropping the columns that helped create the new column
    dataset = dataset.drop(['earliest_cr_line_dt', 'earliest_cr_line'], axis=1)
    
    # Last Credit Pulled Date
    # dropping the column bc it is used post-funding
    dataset = dataset.drop(['last_credit_pull_d'], axis=1)
    
    # Fico Score
    dataset['fico_score'] = (dataset['fico_range_high'] + dataset['fico_range_low'])/2
    dataset['sec_app_fico_score'] = (dataset['sec_app_fico_range_high'] +\
                                       dataset['sec_app_fico_range_low'])/2
    dataset = dataset.drop(['fico_range_high', 'fico_range_low',
                                        'sec_app_fico_range_high',
                                        'sec_app_fico_range_low'], axis=1)
    # Verification status
    # if the income is verified, change to 1. If not verified, change to 0
    dataset.loc[((dataset['verification_status']=='Source Verified') |\
                      (dataset['verification_status']=='Verified')), 'verification_status'] = 1
    dataset.loc[dataset['verification_status']=='Not Verified', 'verification_status'] = 0
    
    # change dtype to int
    dataset['verification_status'] = dataset['verification_status'].astype(int)
    
    dataset.loc[dataset['application_type']=='Joint App', 'application_type'] = 2
    dataset.loc[dataset['application_type']=='Individual', 'application_type'] = 1
    dataset['application_type'] = dataset['application_type'].astype(int)
    
    '''
    for verification_status_joint where there are values in cols that start with
    sec_app, but verification_status_joint is null
    '''
    # add in 'not verified' where applicable
    dataset.loc[((dataset['verification_status_joint'].notnull()) &\
                       (dataset['application_type']==1)),
                      'verification_status_joint'] = 'Not Verified'
    dataset.loc[((dataset['verification_status_joint'].isnull()) &\
                   (dataset['application_type']==2)),
                  'verification_status_joint'] = 'Not Verified'
    dataset.loc[((dataset['verification_status_joint'].isnull()) &\
                   (dataset['application_type']==1)),
                  'verification_status_joint'] = 'Individual'
    
    # if the income is verified, change to 1. If not verified, change to 0
    dataset.loc[((dataset['verification_status_joint']=='Source Verified') |\
                      (dataset['verification_status_joint']=='Verified')),
                      'verification_status_joint'] = 1
    dataset.loc[dataset['verification_status_joint']=='Individual',
                      'verification_status_joint'] = -1
    dataset.loc[dataset['verification_status_joint']=='Not Verified',
                      'verification_status_joint'] = 0
    
    dataset['verification_status_joint'] = dataset['verification_status_joint'].astype(int)
    
    # Reducing Income if not Verified
    dataset['annual_inc_reduced'] = dataset['annual_inc']
    dataset.loc[dataset['verification_status']==0,
                      'annual_inc_reduced'] = dataset['annual_inc'] * 0.7
    dataset['annual_inc_joint_reduced'] = dataset['annual_inc_joint']
    dataset.loc[dataset['verification_status_joint']==0,
                      'annual_inc_joint_reduced'] = dataset['annual_inc_joint'] * 0.7
    
    # Joint Earliest Credit Line
    # converting column to datetime
    dataset['sec_app_earliest_cr_line_dt'] = pd.DatetimeIndex(dataset['sec_app_earliest_cr_line'])

    # creating a new column that will show the number of months
    #that credit has existed for joint customer
    dataset['sec_app_duration_of_credit_months']=(pd.DatetimeIndex(dataset['issue_d']).year\
                                                - pd.DatetimeIndex(dataset['sec_app_earliest_cr_line']).year)*12\
    + (pd.DatetimeIndex(dataset['issue_d']).month - \
       pd.DatetimeIndex(dataset['sec_app_earliest_cr_line']).month)

    # converting null values to -1
    dataset.loc[dataset['sec_app_duration_of_credit_months'].isnull(),
                      'sec_app_duration_of_credit_months'] = -1
    
    dataset = dataset.drop(['sec_app_earliest_cr_line_dt', 'sec_app_earliest_cr_line',
                                    'issue_d', 
                                    'sec_app_earliest_cr_line',
                                    'sec_app_earliest_cr_line_dt'], axis=1)
    
    cols_w_mths_since = [col for col in dataset.columns if col.startswith('mths_since_')]
    for col in cols_w_mths_since:
        dataset.loc[dataset[col].isnull(), col] = 10000
    
    # Get cols with null values
    cols_w_null = []

    # function that gives list of columns that contain null values
    null_cols(dataset, cols_w_null)

    # Fill in null values
    # creates a list of the index where the column == 0
    list(dataset[dataset['emp_length']==0].index)
    
    # getting the columns where there are 0s
    cols_w_0s_and_null = []
    for col in cols_w_null:
        # if the list is not empty, then there are values == 0
        if list(dataset[dataset[col]==0].index)!=[]:
            cols_w_0s_and_null.append(col)
        for col_0 in cols_w_0s_and_null:
            dataset.loc[dataset[col_0].isnull(), col_0] = -1
        # rerun to get new cols w null values

    cols_w_null = []
    # function that gives list of columns that contain null values
    null_cols(dataset, cols_w_null)

    # Employment
    dataset.loc[dataset['emp_title'].isnull(), 'emp_title'] = 'Unknown'
    
    # Employment length
    emplength_mean = dataset['emp_length'].mean()
    dataset.loc[(dataset['emp_length'].isnull()) &\
                  (dataset['emp_title'].notnull()), 'emp_length'] = emplength_mean
    dataset.loc[(dataset['emp_length'].isnull()) &\
                  (dataset['emp_title'].isnull()), 'emp_length'] = -1
    dataset = dataset.drop('emp_title', axis=1)
    
    # joint
    # if no secondary app, convert to -1
    dataset.loc[dataset['annual_inc_joint'].isnull(), 'annual_inc_joint'] = -1
    dataset.loc[dataset['annual_inc_joint_reduced'].isnull(),'annual_inc_joint_reduced'] = -1
    dataset.loc[dataset['sec_app_fico_score'].isnull(), 'sec_app_fico_score'] = -1
    
    # others
    dataset.loc[dataset['annual_inc'].isnull(), 'annual_inc'] = -1
    dataset.loc[dataset['mo_sin_old_rev_tl_op'].isnull(), 'mo_sin_old_rev_tl_op'] = 10000
    dataset.loc[dataset['duration_of_credit_months'].isnull(),
                      'duration_of_credit_months'] = -1
    dataset.loc[dataset['sec_app_duration_of_credit_months'].isnull(),
                      'sec_app_duration_of_credit_months'] = -1
    
    # total accounts
    cols_w_acc = [col for col in dataset.columns if 'acc' in col]
    
    # using the open_acc as a filler for the null values for total_acc
    dataset.loc[dataset['total_acc'].isnull(), 'total_acc'] = dataset['open_acc']
    
    cols_w_null = []
    # function that gives list of columns that contain null values
    null_cols(dataset, cols_w_null)
    cols_w_null_list = list(dataset[cols_w_null].columns)
    
    # getting the columns where there are 0s
    for col in cols_w_null_list:
        dataset.loc[dataset[col].isnull(), col] = -1
        
    cols_w_null = []
    # function that gives list of columns that contain null values
    null_cols(dataset, cols_w_null)
    
    num_cols = [col for col in dataset.columns if '2' in col]
    num_cols.remove('grouped_subgrade_12_345')
    num_cols.remove('grouped_subgrade_123_45')
    dataset = dataset.drop(num_cols, axis=1)
    
    # Income : * Ratios
    dataset[(dataset['application_type']==1) &\
                  (dataset['annual_inc_joint']==dataset['annual_inc'])]\
    [['annual_inc_joint', 'annual_inc']]
    
    # Giving weight to loan amount
    # creating a new column
    dataset['weighted_loan_amnt'] = dataset['loan_amnt']

    # multiplying the new column by its perceived rank according to FICO score ranges
    dataset.loc[dataset['fico_score']<=560,
                  'weighted_loan_amnt'] = dataset['weighted_loan_amnt'] * 5
    dataset.loc[((dataset['fico_score']>560) &\
                   (dataset['fico_score']<=650)),
                  'weighted_loan_amnt'] = dataset['weighted_loan_amnt'] * 4
    dataset.loc[((dataset['fico_score']>650) &\
                   (dataset['fico_score']<=700)),
                  'weighted_loan_amnt'] = dataset['weighted_loan_amnt'] * 3
    dataset.loc[((dataset['fico_score']>700) &\
                   (dataset['fico_score']<=750)),
                  'weighted_loan_amnt'] = dataset['weighted_loan_amnt'] * 2
    dataset.loc[dataset['fico_score']>750,
                  'weighted_loan_amnt'] = dataset['weighted_loan_amnt'] * 1
    dataset['weighted_loan_amnt']
    
    # Giving weight to funded amount
    # creating a new column
    dataset['weighted_funded_amnt'] = dataset['funded_amnt']

    # multiplying the new column by its perceived rank according to FICO score ranges
    dataset.loc[dataset['fico_score']<=560,
                  'weighted_funded_amnt'] = dataset['weighted_funded_amnt'] * 5
    dataset.loc[((dataset['fico_score']>560) &\
                   (dataset['fico_score']<=650)),
                  'weighted_funded_amnt'] = dataset['weighted_funded_amnt'] * 4
    dataset.loc[((dataset['fico_score']>650) &\
                   (dataset['fico_score']<=700)),
                  'weighted_funded_amnt'] = dataset['weighted_funded_amnt'] * 3
    dataset.loc[((dataset['fico_score']>700) &\
                   (dataset['fico_score']<=750)),
                  'weighted_funded_amnt'] = dataset['weighted_funded_amnt'] * 2
    dataset.loc[dataset['fico_score']>750,
                  'weighted_funded_amnt'] = dataset['weighted_funded_amnt'] * 1
    dataset['weighted_funded_amnt']
    
    # Giving weight to installment
    # creating a new column
    dataset['weighted_installment'] = dataset['installment']

    # multiplying the new column by its perceived rank according to FICO score ranges
    dataset.loc[dataset['fico_score']<=560,
                  'weighted_installment'] = dataset['weighted_installment'] * 5
    dataset.loc[((dataset['fico_score']>560) &\
                   (dataset['fico_score']<=650)),
                  'weighted_installment'] = dataset['weighted_installment'] * 4
    dataset.loc[((dataset['fico_score']>650) &\
                   (dataset['fico_score']<=700)),
                  'weighted_installment'] = dataset['weighted_installment'] * 3
    dataset.loc[((dataset['fico_score']>700) &\
                   (dataset['fico_score']<=750)),
                  'weighted_installment'] = dataset['weighted_installment'] * 2
    dataset.loc[dataset['fico_score']>750,
                  'weighted_installment'] = dataset['weighted_installment'] * 1
    dataset['weighted_installment']
    
    # 1st borrower income
    # Income to loan amount ratio
    dataset['ratio_inc_loan'] = dataset['annual_inc'] / dataset['loan_amnt']
    dataset['ratio_inc_weightedloan'] = dataset['annual_inc'] / dataset['weighted_loan_amnt']

    # income reduced to loan amount ratio
    dataset['ratio_redinc_loan'] = dataset['annual_inc_reduced'] / dataset['loan_amnt']
    dataset['ratio_redinc_weightedloan'] = dataset['annual_inc_reduced'] /\
    dataset['weighted_loan_amnt']

    # Income to funded amount ratio
    dataset['ratio_inc_funded'] = dataset['annual_inc'] / dataset['funded_amnt']
    dataset['ratio_inc_weightedfunded'] = dataset['annual_inc'] / dataset['weighted_funded_amnt']

    # income reduced to funded amount ratio
    dataset['ratio_redinc_funded'] = dataset['annual_inc_reduced'] /\
    dataset['funded_amnt']
    dataset['ratio_redinc_weightedfunded'] = dataset['annual_inc_reduced'] /\
    dataset['weighted_funded_amnt']

    # installment to income ratio
    dataset['ratio_install_inc'] = dataset['annual_inc'] / dataset['installment']
    dataset['ratio_weightedinstall_inc'] = dataset['annual_inc'] / dataset['weighted_installment']

    # installment to reduced income ratio
    dataset['ratio_install_redinc'] = dataset['annual_inc_reduced'] /\
    dataset['installment']
    dataset['ratio_weightedinstall_redinc'] = dataset['annual_inc_reduced'] /\
    dataset['weighted_installment']
    
    # joint borrower income
    # Income to loan amount ratio
    dataset['ratio_incjoint_loan'] = dataset['annual_inc_joint'] / dataset['loan_amnt']
    dataset['ratio_incjoint_weightedloan'] = dataset['annual_inc_joint'] /\
    dataset['weighted_loan_amnt']

    # income reduced to loan amount ratio
    dataset['ratio_redincjoint_loan'] = dataset['annual_inc_joint_reduced'] /\
    dataset['loan_amnt']
    dataset['ratio_redincjoint_weightedloan'] = dataset['annual_inc_joint_reduced'] /\
    dataset['weighted_loan_amnt']

    # Income to funded amount ratio
    dataset['ratio_incjoint_funded'] = dataset['annual_inc_joint'] /\
    dataset['funded_amnt']
    dataset['ratio_incjoint_weightedfunded'] = dataset['annual_inc_joint'] /\
    dataset['weighted_funded_amnt']

    # income reduced to funded amount ratio
    dataset['ratio_redincjoint_funded'] = dataset['annual_inc_joint_reduced'] /\
    dataset['funded_amnt']
    dataset['ratio_redincjoint_weightedfunded'] = dataset['annual_inc_joint_reduced'] /\
    dataset['weighted_funded_amnt']

    # Income to installment ratio
    dataset['ratio_incjoint_installment'] = dataset['annual_inc_joint']/\
    dataset['installment']
    dataset['ratio_incjoint_weightedinstallment'] = dataset['annual_inc_joint']/\
    dataset['weighted_installment']

    # Reduced income to installment ratio
    dataset['ratio_redincjoint_installment'] = dataset['annual_inc_joint_reduced']/\
    dataset['installment']
    dataset['ratio_redincjoint_weightedinstallment'] = dataset['annual_inc_joint_reduced']/\
    dataset['weighted_installment']
    
    # Loan Amount vs Funded Amount
    dataset['loan_minus_funded'] = dataset['loan_amnt'] - dataset['funded_amnt']
    dataset['ratio_loan_funded'] = dataset['loan_amnt'] / dataset['funded_amnt']

In [48]:
acc_clean_funded

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,purpose,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,total_pymnt,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_act_il,mths_since_rcnt_il,total_bal_il,il_util,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_mths_since_last_major_derog,roi,principal_70,total_expected,total_expected_70,grouped_grade_NO_EG,grouped_subgrade_12_345,grouped_subgrade_123_45,grouped_subgrade,earliest_cr_line_dt,duration_of_credit_months
0,3600.0,3600.0,3600.0,36,13.99,123.03,3,14,leadman,10,2,55000.0,Not Verified,Dec-2015,Fully Paid,debt_consolidation,5.91,0.0,Aug-2003,675.0,679.0,30.0,NaN,7.0,0.0,2765.0,29.7,13.0,w,4421.723917,Mar-2019,0.0,30.0,1.0,Individual,NaN,NaN,NaN,0.0,722.0,144904.0,2.0,21.0,4981.0,36.0,722.0,34.0,9300.0,3.0,1.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.228257,2520.0,4429.08,3100.36,3,6,6,14,2003-08-01,148.0
1,20000.0,20000.0,20000.0,36,9.17,637.58,2,7,Vice President of Recruiting Operations,10,2,180000.0,Not Verified,Dec-2015,Fully Paid,debt_consolidation,14.67,0.0,Jun-1990,680.0,684.0,49.0,NaN,12.0,0.0,87329.0,84.5,27.0,f,21393.800000,Mar-2019,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,360358.0,2.0,18.0,29433.0,63.0,13048.0,74.0,94200.0,1.0,0.0,6.0,30030.0,0.0,102.9,0.0,0.0,142.0,306.0,10.0,10.0,4.0,12.0,NaN,10.0,NaN,4.0,6.0,4.0,5.0,7.0,9.0,16.0,6.0,12.0,96.3,100.0,0.0,0.0,388852.0,116762.0,31500.0,46452.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.069690,14000.0,22952.88,16067.02,2,3,3,7,1990-06-01,306.0
2,20000.0,20000.0,20000.0,36,8.49,631.26,2,6,road driver,10,2,85000.0,Not Verified,Dec-2015,Fully Paid,major_purchase,17.61,1.0,Feb-1999,705.0,709.0,3.0,NaN,8.0,0.0,826.0,5.7,15.0,w,21538.508977,Mar-2019,0.0,3.0,1.0,Individual,NaN,NaN,NaN,0.0,0.0,141601.0,3.0,13.0,27111.0,75.0,640.0,55.0,14500.0,1.0,0.0,4.0,17700.0,13674.0,5.7,0.0,0.0,149.0,55.0,32.0,13.0,3.0,32.0,NaN,8.0,NaN,2.0,2.0,3.0,3.0,9.0,3.0,3.0,2.0,8.0,93.3,0.0,0.0,0.0,193390.0,27937.0,14500.0,36144.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.076925,14000.0,22725.36,15907.75,2,3,3,6,1999-02-01,202.0
3,10000.0,10000.0,10000.0,36,6.49,306.45,1,2,SERVICE MANAGER,6,0,85000.0,Not Verified,Dec-2015,Fully Paid,credit_card,13.07,0.0,Apr-2002,685.0,689.0,NaN,106.0,14.0,1.0,10464.0,34.5,23.0,w,10998.971575,Mar-2019,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,8341.0,27957.0,1.0,35.0,17493.0,57.0,2524.0,46.0,30300.0,2.0,0.0,7.0,1997.0,8182.0,50.1,0.0,0.0,164.0,129.0,1.0,1.0,1.0,4.0,NaN,1.0,NaN,6.0,9.0,7.0,10.0,3.0,13.0,19.0,9.0,14.0,95.7,28.6,1.0,0.0,61099.0,27957.0,16400.0,30799.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.099897,7000.0,11032.20,7722.54,1,1,1,2,2002-04-01,164.0
4,8000.0,8000.0,8000.0,36,11.48,263.74,2,10,

In [49]:
acc_clean_70total

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,purpose,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,total_pymnt,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_act_il,mths_since_rcnt_il,total_bal_il,il_util,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_mths_since_last_major_derog,roi,principal_70,total_expected,total_expected_70,grouped_grade_NO_EG,grouped_subgrade_12_345,grouped_subgrade_123_45,grouped_subgrade,earliest_cr_line_dt,duration_of_credit_months
0,3600.0,3600.0,3600.0,36,13.99,123.03,3,14,leadman,10,2,55000.0,Not Verified,Dec-2015,Fully Paid,debt_consolidation,5.91,0.0,Aug-2003,675.0,679.0,30.0,NaN,7.0,0.0,2765.0,29.7,13.0,w,4421.723917,Mar-2019,0.0,30.0,1.0,Individual,NaN,NaN,NaN,0.0,722.0,144904.0,2.0,21.0,4981.0,36.0,722.0,34.0,9300.0,3.0,1.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.228257,2520.0,4429.08,3100.36,3,6,6,14,2003-08-01,148.0
1,20000.0,20000.0,20000.0,36,9.17,637.58,2,7,Vice President of Recruiting Operations,10,2,180000.0,Not Verified,Dec-2015,Fully Paid,debt_consolidation,14.67,0.0,Jun-1990,680.0,684.0,49.0,NaN,12.0,0.0,87329.0,84.5,27.0,f,21393.800000,Mar-2019,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,360358.0,2.0,18.0,29433.0,63.0,13048.0,74.0,94200.0,1.0,0.0,6.0,30030.0,0.0,102.9,0.0,0.0,142.0,306.0,10.0,10.0,4.0,12.0,NaN,10.0,NaN,4.0,6.0,4.0,5.0,7.0,9.0,16.0,6.0,12.0,96.3,100.0,0.0,0.0,388852.0,116762.0,31500.0,46452.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.069690,14000.0,22952.88,16067.02,2,3,3,7,1990-06-01,306.0
2,20000.0,20000.0,20000.0,36,8.49,631.26,2,6,road driver,10,2,85000.0,Not Verified,Dec-2015,Fully Paid,major_purchase,17.61,1.0,Feb-1999,705.0,709.0,3.0,NaN,8.0,0.0,826.0,5.7,15.0,w,21538.508977,Mar-2019,0.0,3.0,1.0,Individual,NaN,NaN,NaN,0.0,0.0,141601.0,3.0,13.0,27111.0,75.0,640.0,55.0,14500.0,1.0,0.0,4.0,17700.0,13674.0,5.7,0.0,0.0,149.0,55.0,32.0,13.0,3.0,32.0,NaN,8.0,NaN,2.0,2.0,3.0,3.0,9.0,3.0,3.0,2.0,8.0,93.3,0.0,0.0,0.0,193390.0,27937.0,14500.0,36144.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.076925,14000.0,22725.36,15907.75,2,3,3,6,1999-02-01,202.0
3,10000.0,10000.0,10000.0,36,6.49,306.45,1,2,SERVICE MANAGER,6,0,85000.0,Not Verified,Dec-2015,Fully Paid,credit_card,13.07,0.0,Apr-2002,685.0,689.0,NaN,106.0,14.0,1.0,10464.0,34.5,23.0,w,10998.971575,Mar-2019,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,8341.0,27957.0,1.0,35.0,17493.0,57.0,2524.0,46.0,30300.0,2.0,0.0,7.0,1997.0,8182.0,50.1,0.0,0.0,164.0,129.0,1.0,1.0,1.0,4.0,NaN,1.0,NaN,6.0,9.0,7.0,10.0,3.0,13.0,19.0,9.0,14.0,95.7,28.6,1.0,0.0,61099.0,27957.0,16400.0,30799.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.099897,7000.0,11032.20,7722.54,1,1,1,2,2002-04-01,164.0
4,8000.0,8000.0,8000.0,36,11.48,263.74,2,10,

In [50]:
acc_clean_70prcp

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,purpose,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,total_pymnt,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_act_il,mths_since_rcnt_il,total_bal_il,il_util,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_mths_since_last_major_derog,roi,principal_70,total_expected,total_expected_70,grouped_grade_NO_EG,grouped_subgrade_12_345,grouped_subgrade_123_45,grouped_subgrade,earliest_cr_line_dt,duration_of_credit_months
0,3600.0,3600.0,3600.0,36,13.99,123.03,3,14,leadman,10,2,55000.0,Not Verified,Dec-2015,Fully Paid,debt_consolidation,5.91,0.0,Aug-2003,675.0,679.0,30.0,NaN,7.0,0.0,2765.0,29.7,13.0,w,4421.723917,Mar-2019,0.0,30.0,1.0,Individual,NaN,NaN,NaN,0.0,722.0,144904.0,2.0,21.0,4981.0,36.0,722.0,34.0,9300.0,3.0,1.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.228257,2520.0,4429.08,3100.36,3,6,6,14,2003-08-01,148.0
1,20000.0,20000.0,20000.0,36,9.17,637.58,2,7,Vice President of Recruiting Operations,10,2,180000.0,Not Verified,Dec-2015,Fully Paid,debt_consolidation,14.67,0.0,Jun-1990,680.0,684.0,49.0,NaN,12.0,0.0,87329.0,84.5,27.0,f,21393.800000,Mar-2019,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,360358.0,2.0,18.0,29433.0,63.0,13048.0,74.0,94200.0,1.0,0.0,6.0,30030.0,0.0,102.9,0.0,0.0,142.0,306.0,10.0,10.0,4.0,12.0,NaN,10.0,NaN,4.0,6.0,4.0,5.0,7.0,9.0,16.0,6.0,12.0,96.3,100.0,0.0,0.0,388852.0,116762.0,31500.0,46452.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.069690,14000.0,22952.88,16067.02,2,3,3,7,1990-06-01,306.0
2,20000.0,20000.0,20000.0,36,8.49,631.26,2,6,road driver,10,2,85000.0,Not Verified,Dec-2015,Fully Paid,major_purchase,17.61,1.0,Feb-1999,705.0,709.0,3.0,NaN,8.0,0.0,826.0,5.7,15.0,w,21538.508977,Mar-2019,0.0,3.0,1.0,Individual,NaN,NaN,NaN,0.0,0.0,141601.0,3.0,13.0,27111.0,75.0,640.0,55.0,14500.0,1.0,0.0,4.0,17700.0,13674.0,5.7,0.0,0.0,149.0,55.0,32.0,13.0,3.0,32.0,NaN,8.0,NaN,2.0,2.0,3.0,3.0,9.0,3.0,3.0,2.0,8.0,93.3,0.0,0.0,0.0,193390.0,27937.0,14500.0,36144.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.076925,14000.0,22725.36,15907.75,2,3,3,6,1999-02-01,202.0
3,10000.0,10000.0,10000.0,36,6.49,306.45,1,2,SERVICE MANAGER,6,0,85000.0,Not Verified,Dec-2015,Fully Paid,credit_card,13.07,0.0,Apr-2002,685.0,689.0,NaN,106.0,14.0,1.0,10464.0,34.5,23.0,w,10998.971575,Mar-2019,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,8341.0,27957.0,1.0,35.0,17493.0,57.0,2524.0,46.0,30300.0,2.0,0.0,7.0,1997.0,8182.0,50.1,0.0,0.0,164.0,129.0,1.0,1.0,1.0,4.0,NaN,1.0,NaN,6.0,9.0,7.0,10.0,3.0,13.0,19.0,9.0,14.0,95.7,28.6,1.0,0.0,61099.0,27957.0,16400.0,30799.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.099897,7000.0,11032.20,7722.54,1,1,1,2,2002-04-01,164.0
4,8000.0,8000.0,8000.0,36,11.48,263.74,2,10,

In [51]:
target_3_train_funded

,investor_expect,investor_expect_70total,investor_expect_70principal
0,0,0,0
1,0,0,0
2,0,0,0
3,0,0,0
4,0,0,0
...,...,...,...
674044,1,1,1
674045,1,1,1
674046,1,1,1
674047,1,1,1


In [52]:
target_3_train_70total

,investor_expect,investor_expect_70total,investor_expect_70principal
0,0,0,0
1,0,0,0
2,0,0,0
3,0,0,0
4,0,0,0
...,...,...,...
674044,1,1,1
674045,1,1,1
674046,1,1,1
674047,1,1,1


In [53]:
target_3_train_70principal

,investor_expect,investor_expect_70total,investor_expect_70principal
0,0,0,0
1,0,0,0
2,0,0,0
3,0,0,0
4,0,0,0
...,...,...,...
674044,1,1,1
674045,1,1,1
674046,1,1,1
674047,1,1,1


# Dummification

In [112]:
# getting a list of cols w dtype object

cols_w_obj = list(acc_clean.select_dtypes(include=['object']).columns)

cols_w_obj

['loan_status', 'purpose', 'initial_list_status']

In [113]:
acc_clean.loc[acc_clean['initial_list_status']=='w', 'initial_list_status'] = 0
acc_clean.loc[acc_clean['initial_list_status']=='f', 'initial_list_status'] = 1

acc_clean[['initial_list_status']] = acc_clean[['initial_list_status']].astype(int)
acc_clean[['initial_list_status']]

,initial_list_status
0,0
1,1
2,0
3,0
4,0
...,...
674044,1
674045,1
674046,1
674047,1


In [114]:
acc_clean['loan_status'].unique()

array(['Fully Paid', 'Charged Off', 'Default',
       'Does not meet the credit policy. Status:Fully Paid',
       'Does not meet the credit policy. Status:Charged Off'],
      dtype=object)

In [115]:
acc_clean.loc[acc_clean['loan_status']=='Fully Paid', 'loan_status'] = 5
acc_clean.loc[acc_clean['loan_status']=='Charged Off', 'loan_status'] = 0
acc_clean.loc[acc_clean['loan_status']=='Current', 'loan_status'] = 4
acc_clean.loc[acc_clean['loan_status']=='Late (31-120 days)', 'loan_status'] = 3
acc_clean.loc[acc_clean['loan_status']=='In Grace Period', 'loan_status'] = 1
acc_clean.loc[acc_clean['loan_status']=='Late (16-30 days)', 'loan_status'] = 2
acc_clean.loc[acc_clean['loan_status']=='Default', 'loan_status'] = 0
acc_clean.loc[acc_clean['loan_status']=='Does not meet the credit policy. Status:Fully Paid',\
              'loan_status'] = 5
acc_clean.loc[acc_clean['loan_status']=='Does not meet the credit policy. Status:Charged Off',\
              'loan_status'] = 0

acc_clean['loan_status'] = acc_clean['loan_status'].astype(int)

acc_clean[['loan_status']]

,loan_status
0,5
1,5
2,5
3,5
4,5
...,...
674044,0
674045,0
674046,0
674047,0


In [116]:
acc_clean = acc_clean.drop('loan_status', axis=1)

acc_clean

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,purpose,dti,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,total_pymnt,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_act_il,mths_since_rcnt_il,total_bal_il,il_util,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,avg_cur_bal,bc_open_to_buy,bc_util,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_mths_since_last_major_derog,roi,principal_70,total_expected,total_expected_70,grouped_grade_NO_EG,grouped_subgrade_12_345,grouped_subgrade_123_45,grouped_subgrade,duration_of_credit_months,fico_score,sec_app_fico_score,annual_inc_reduced,annual_inc_joint_reduced,sec_app_duration_of_credit_months,weighted_loan_amnt,weighted_funded_amnt,weighted_installment,ratio_inc_loan,ratio_inc_weightedloan,ratio_redinc_loan,ratio_redinc_weightedloan,ratio_inc_funded,ratio_inc_weightedfunded,ratio_redinc_funded,ratio_redinc_weightedfunded,ratio_install_inc,ratio_weightedinstall_inc,ratio_install_redinc,ratio_weightedinstall_redinc,ratio_incjoint_loan,ratio_incjoint_weightedloan,ratio_redincjoint_loan,ratio_redincjoint_weightedloan,ratio_incjoint_funded,ratio_incjoint_weightedfunded,ratio_redincjoint_funded,ratio_redincjoint_weightedfunded,ratio_incjoint_installment,ratio_incjoint_weightedinstallment,ratio_redincjoint_installment,ratio_redincjoint_weightedinstallment,loan_minus_funded,ratio_loan_funded
0,3600.0,3600.0,3600.0,36,13.99,123.03,3,14,10.0,2,55000.0,0,debt_consolidation,5.91,30.0,10000.0,7.0,0.0,2765.0,29.7,13.0,0,4421.723917,30.0,1.0,1,-1.0,-1.0,-1,0.0,722.0,144904.0,2.0,21.0,4981.0,36.0,722.0,34.0,9300.0,3.0,1.0,20701.0,1506.0,37.2,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.228257,2520.0,4429.08,3100.36,3,6,6,14,148.0,677.0,-1.0,38500.0,-1.0,-1.0,10800.0,10800.0,369.09,15.277778,5.092593,10.694444,3.564815,15.277778,5.092593,10.694444,3.564815,447.045436,149.015145,312.931805,104.310602,-0.000278,-0.000093,-0.000278,-0.000093,-0.000278,-0.000093,-0.000278,-0.000093,-0.008128,-0.002709,-0.008128,-0.002709,0.0,1.0
1,20000.0,20000.0,20000.0,36,9.17,637.58,2,7,10.0,2,180000.0,0,debt_consolidation,14.67,49.0,10000.0,12.0,0.0,87329.0,84.5,27.0,1,21393.800000,10000.0,1.0,1,-1.0,-1.0,-1,0.0,0.0,360358.0,2.0,18.0,29433.0,63.0,13048.0,74.0,94200.0,1.0,0.0,30030.0,0.0,102.9,0.0,142.0,306.0,10.0,10.0,4.0,12.0,10000.0,10.0,10000.0,4.0,6.0,4.0,5.0,7.0,9.0,16.0,6.0,12.0,96.3,100.0,0.0,0.0,388852.0,116762.0,31500.0,46452.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.069690,14000.0,22952.88,16067.02,2,3,3,7,306.0,682.0,-1.0,126000.0,-1.0,-1.0,60000.0,60000.0,1912.74,9.000000,3.000000,6.300000,2.100000,9.000000,3.000000,6.300000,2.100000,282.317513,94.105838,197.622259,65.874086,-0.000050,-0.000017,-0.000050,-0.000017,-0.000050,-0.000017,-0.000050,-0.000017,-0.001568,-0.000523,-0.001568,-0.000523,0.0,1.0
2,20000.0,20000.0,20000.0,36,8.49,631.26,2,6,10.0,2,85000.0,0,major_purchase,17.61,3.0,10000.0,8.0,0.0,826.0,5.7,15.0,0,21538.508977,3.0,1.0,1,-1.0,-1.0,-1,0.0,0.0,141601.0,3.0,13.0,27111.0,75.0,640.0,55.0,14500.0,1.0,0.0,17700.0,13674.0,5.7,0.0,149.0,55.0,32.0,13.0,3.0,32.0,100

In [117]:
# dummification

acc_dum = pd.get_dummies(acc_clean, drop_first=True)

acc_dum

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,dti,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,total_pymnt,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_act_il,mths_since_rcnt_il,total_bal_il,il_util,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,avg_cur_bal,bc_open_to_buy,bc_util,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_mths_since_last_major_derog,roi,principal_70,total_expected,total_expected_70,grouped_grade_NO_EG,grouped_subgrade_12_345,grouped_subgrade_123_45,grouped_subgrade,duration_of_credit_months,fico_score,sec_app_fico_score,annual_inc_reduced,annual_inc_joint_reduced,sec_app_duration_of_credit_months,weighted_loan_amnt,weighted_funded_amnt,weighted_installment,ratio_inc_loan,ratio_inc_weightedloan,ratio_redinc_loan,ratio_redinc_weightedloan,ratio_inc_funded,ratio_inc_weightedfunded,ratio_redinc_funded,ratio_redinc_weightedfunded,ratio_install_inc,ratio_weightedinstall_inc,ratio_install_redinc,ratio_weightedinstall_redinc,ratio_incjoint_loan,ratio_incjoint_weightedloan,ratio_redincjoint_loan,ratio_redincjoint_weightedloan,ratio_incjoint_funded,ratio_incjoint_weightedfunded,ratio_redincjoint_funded,ratio_redincjoint_weightedfunded,ratio_incjoint_installment,ratio_incjoint_weightedinstallment,ratio_redincjoint_installment,ratio_redincjoint_weightedinstallment,loan_minus_funded,ratio_loan_funded,purpose_credit_card,purpose_debt_consolidation,purpose_educational,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding
0,3600.0,3600.0,3600.0,36,13.99,123.03,3,14,10.0,2,55000.0,0,5.91,30.0,10000.0,7.0,0.0,2765.0,29.7,13.0,0,4421.723917,30.0,1.0,1,-1.0,-1.0,-1,0.0,722.0,144904.0,2.0,21.0,4981.0,36.0,722.0,34.0,9300.0,3.0,1.0,20701.0,1506.0,37.2,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.228257,2520.0,4429.08,3100.36,3,6,6,14,148.0,677.0,-1.0,38500.0,-1.0,-1.0,10800.0,10800.0,369.09,15.277778,5.092593,10.694444,3.564815,15.277778,5.092593,10.694444,3.564815,447.045436,149.015145,312.931805,104.310602,-0.000278,-0.000093,-0.000278,-0.000093,-0.000278,-0.000093,-0.000278,-0.000093,-0.008128,-0.002709,-0.008128,-0.002709,0.0,1.0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,20000.0,20000.0,20000.0,36,9.17,637.58,2,7,10.0,2,180000.0,0,14.67,49.0,10000.0,12.0,0.0,87329.0,84.5,27.0,1,21393.800000,10000.0,1.0,1,-1.0,-1.0,-1,0.0,0.0,360358.0,2.0,18.0,29433.0,63.0,13048.0,74.0,94200.0,1.0,0.0,30030.0,0.0,102.9,0.0,142.0,306.0,10.0,10.0,4.0,12.0,10000.0,10.0,10000.0,4.0,6.0,4.0,5.0,7.0,9.0,16.0,6.0,12.0,96.3,100.0,0.0,0.0,388852.0,116762.0,31500.0,46452.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.069690,14000.0,22952.88,16067.02,2,3,3,7,306.0,682.0,-1.0,126000.0,-1.0,-1.0,60000.0,60000.0,1912.74,9.000000,3.000000,6.300000,2.100000,9.000000,3.000000,6.300000,2.100000,282.317513,94.105838,197.622259,65.874086,-0.000050,-0.000017,-0.000050,-0.000017,-0.000050,-0.000017,-0.000050,-0.000017,-0.001568,-0.000523,-0.001568,-0.000523,0.0,1.0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,20000.0,20000.0,

# Scaling the dataset

In [118]:
# we are preserving total_pymnt to use as our 

acc_totalpayment = copy.deepcopy(acc_dum[['total_pymnt']])

acc_dum = acc_dum.drop('total_pymnt', axis=1)

In [119]:
# creating copies

import copy

acc_full = copy.deepcopy(acc_dum)

In [120]:
# full scaling

from sklearn.preprocessing import MinMaxScaler

acc_dum_cols = list(acc_dum.columns)

for col in acc_dum_cols:
    acc_full[col] = MinMaxScaler().fit_transform(np.array(acc_full[col]).reshape(-1,1))
    
acc_full

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,dti,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_act_il,mths_since_rcnt_il,total_bal_il,il_util,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,avg_cur_bal,bc_open_to_buy,bc_util,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_mths_since_last_major_derog,roi,principal_70,total_expected,total_expected_70,grouped_grade_NO_EG,grouped_subgrade_12_345,grouped_subgrade_123_45,grouped_subgrade,duration_of_credit_months,fico_score,sec_app_fico_score,annual_inc_reduced,annual_inc_joint_reduced,sec_app_duration_of_credit_months,weighted_loan_amnt,weighted_funded_amnt,weighted_installment,ratio_inc_loan,ratio_inc_weightedloan,ratio_redinc_loan,ratio_redinc_weightedloan,ratio_inc_funded,ratio_inc_weightedfunded,ratio_redinc_funded,ratio_redinc_weightedfunded,ratio_install_inc,ratio_weightedinstall_inc,ratio_install_redinc,ratio_weightedinstall_redinc,ratio_incjoint_loan,ratio_incjoint_weightedloan,ratio_redincjoint_loan,ratio_redincjoint_weightedloan,ratio_incjoint_funded,ratio_incjoint_weightedfunded,ratio_redincjoint_funded,ratio_redincjoint_weightedfunded,ratio_incjoint_installment,ratio_incjoint_weightedinstallment,ratio_redincjoint_installment,ratio_redincjoint_weightedinstallment,loan_minus_funded,ratio_loan_funded,purpose_credit_card,purpose_debt_consolidation,purpose_educational,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding
0,0.078481,0.078481,0.09,0.0,0.338006,0.068867,0.333333,0.382353,1.000000,0.666667,0.005000,0.0,0.00691,0.0030,1.0000,0.094118,0.011494,0.000952,0.034367,0.080460,0.0,0.0030,0.0,0.0,0.0,0.0,0.0,0.2,7.899441e-05,0.028016,0.051724,0.0021,0.002912,0.098143,0.000931,0.189189,0.00093,0.121212,0.038462,0.021608,0.004116,0.112155,0.000004,0.149,0.012603,0.010724,0.012698,0.038462,0.0004,0.0069,0.0004,0.0069,0.083333,0.081967,0.054545,0.084507,0.02500,0.068493,0.077519,0.111111,0.094118,0.771287,0.009901,0.083333,0.011628,0.017805,0.002273,0.002172,0.006535,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.679164,0.078481,0.052441,0.052441,0.50,0.384615,0.384615,0.541667,0.149,0.276008,0.0,0.003500,0.0,0.0,0.085427,0.085427,0.068867,0.002619,0.002619,0.001833,0.001833,0.002619,0.002619,0.001833,0.001833,0.002575,0.002575,0.001803,0.001803,0.000008,0.000007,0.000010,0.000010,0.000008,0.000007,0.000010,0.000010,0.000027,0.000009,0.000037,0.000013,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.493671,0.493671,0.50,0.0,0.150312,0.368914,0.166667,0.176471,1.000000,0.666667,0.016365,0.0,0.01567,0.0049,1.0000,0.152941,0.011494,0.030063,0.095713,0.160920,1.0,1.0000,0.0,0.0,0.0,0.0,0.0,0.2,1.092592e-07,0.069672,0.051724,0.0018,0.017203,0.169761,0.016797,0.405405,0.00942,0.060606,0.019231,0.031345,0.000003,0.305050,0.000004,0.143,0.030406,0.029491,0.034921,0.096154,0.0012,1.0000,0.0010,1.0000,0.138889,0.114754,0.090909,0.084507,0.05000,0.136986,0.131783,0.155556,0.152941,0.963366,1.000000,0.083333,0.011628,0.038885,0.034260,0.028495,0.022100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.591485,0.49

In [121]:
col_names = list(acc_full.columns)
col_names.append('total_pymnt')

acc_full = pd.DataFrame(np.concatenate((acc_full, acc_totalpayment), axis=1),
                             columns=col_names)
acc_full

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,dti,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_act_il,mths_since_rcnt_il,total_bal_il,il_util,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,avg_cur_bal,bc_open_to_buy,bc_util,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_mths_since_last_major_derog,roi,principal_70,total_expected,total_expected_70,grouped_grade_NO_EG,grouped_subgrade_12_345,grouped_subgrade_123_45,grouped_subgrade,duration_of_credit_months,fico_score,sec_app_fico_score,annual_inc_reduced,annual_inc_joint_reduced,sec_app_duration_of_credit_months,weighted_loan_amnt,weighted_funded_amnt,weighted_installment,ratio_inc_loan,ratio_inc_weightedloan,ratio_redinc_loan,ratio_redinc_weightedloan,ratio_inc_funded,ratio_inc_weightedfunded,ratio_redinc_funded,ratio_redinc_weightedfunded,ratio_install_inc,ratio_weightedinstall_inc,ratio_install_redinc,ratio_weightedinstall_redinc,ratio_incjoint_loan,ratio_incjoint_weightedloan,ratio_redincjoint_loan,ratio_redincjoint_weightedloan,ratio_incjoint_funded,ratio_incjoint_weightedfunded,ratio_redincjoint_funded,ratio_redincjoint_weightedfunded,ratio_incjoint_installment,ratio_incjoint_weightedinstallment,ratio_redincjoint_installment,ratio_redincjoint_weightedinstallment,loan_minus_funded,ratio_loan_funded,purpose_credit_card,purpose_debt_consolidation,purpose_educational,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,total_pymnt
0,0.078481,0.078481,0.09,0.0,0.338006,0.068867,0.333333,0.382353,1.000000,0.666667,0.005000,0.0,0.00691,0.0030,1.0000,0.094118,0.011494,0.000952,0.034367,0.080460,0.0,0.0030,0.0,0.0,0.0,0.0,0.0,0.2,7.899441e-05,0.028016,0.051724,0.0021,0.002912,0.098143,0.000931,0.189189,0.00093,0.121212,0.038462,0.021608,0.004116,0.112155,0.000004,0.149,0.012603,0.010724,0.012698,0.038462,0.0004,0.0069,0.0004,0.0069,0.083333,0.081967,0.054545,0.084507,0.02500,0.068493,0.077519,0.111111,0.094118,0.771287,0.009901,0.083333,0.011628,0.017805,0.002273,0.002172,0.006535,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.679164,0.078481,0.052441,0.052441,0.50,0.384615,0.384615,0.541667,0.149,0.276008,0.0,0.003500,0.0,0.0,0.085427,0.085427,0.068867,0.002619,0.002619,0.001833,0.001833,0.002619,0.002619,0.001833,0.001833,0.002575,0.002575,0.001803,0.001803,0.000008,0.000007,0.000010,0.000010,0.000008,0.000007,0.000010,0.000010,0.000027,0.000009,0.000037,0.000013,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4421.723917
1,0.493671,0.493671,0.50,0.0,0.150312,0.368914,0.166667,0.176471,1.000000,0.666667,0.016365,0.0,0.01567,0.0049,1.0000,0.152941,0.011494,0.030063,0.095713,0.160920,1.0,1.0000,0.0,0.0,0.0,0.0,0.0,0.2,1.092592e-07,0.069672,0.051724,0.0018,0.017203,0.169761,0.016797,0.405405,0.00942,0.060606,0.019231,0.031345,0.000003,0.305050,0.000004,0.143,0.030406,0.029491,0.034921,0.096154,0.0012,1.0000,0.0010,1.0000,0.138889,0.114754,0.090909,0.084507,0.05000,0.136986,0.131783,0.155556,0.152941,0.963366,1.000000,0.083333,0.011628,0.038885,0.034260,0.028495,0.022100,0.0,0.0,0.0,0.0,0.0,0

## Downsampling the dataset

In [122]:
target_3_train_downsample[['investor_expect']].value_counts()

investor_expect
0                  548853
1                  125196
dtype: int64

In [123]:
from sklearn.utils import resample

label = target_3_train_downsample['investor_expect']

target_mean = label.mean()

target_fullypaid = label == 0

subsample_ind = resample(label[target_fullypaid].index.values, \
                         replace=False,
                         n_samples=int(len(target_3_train_downsample.index)*target_mean))

combined = np.concatenate([subsample_ind, label[~target_fullypaid].index.values])

acc_subsample = acc_full.iloc[combined]
target_subsample = target_3_train_downsample[['investor_expect']].iloc[combined]

acc_subsample = acc_subsample.reset_index()
acc_subsample = acc_subsample.drop('index', axis=1)

target_subsample = target_subsample.reset_index()
target_subsample = target_subsample.drop('index', axis=1)

target_subsample.mean()

investor_expect    0.5
dtype: float64

In [124]:
acc_subsample

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,dti,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_act_il,mths_since_rcnt_il,total_bal_il,il_util,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,avg_cur_bal,bc_open_to_buy,bc_util,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_mths_since_last_major_derog,roi,principal_70,total_expected,total_expected_70,grouped_grade_NO_EG,grouped_subgrade_12_345,grouped_subgrade_123_45,grouped_subgrade,duration_of_credit_months,fico_score,sec_app_fico_score,annual_inc_reduced,annual_inc_joint_reduced,sec_app_duration_of_credit_months,weighted_loan_amnt,weighted_funded_amnt,weighted_installment,ratio_inc_loan,ratio_inc_weightedloan,ratio_redinc_loan,ratio_redinc_weightedloan,ratio_inc_funded,ratio_inc_weightedfunded,ratio_redinc_funded,ratio_redinc_weightedfunded,ratio_install_inc,ratio_weightedinstall_inc,ratio_install_redinc,ratio_weightedinstall_redinc,ratio_incjoint_loan,ratio_incjoint_weightedloan,ratio_redincjoint_loan,ratio_redincjoint_weightedloan,ratio_incjoint_funded,ratio_incjoint_weightedfunded,ratio_redincjoint_funded,ratio_redincjoint_weightedfunded,ratio_incjoint_installment,ratio_incjoint_weightedinstallment,ratio_redincjoint_installment,ratio_redincjoint_weightedinstallment,loan_minus_funded,ratio_loan_funded,purpose_credit_card,purpose_debt_consolidation,purpose_educational,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,total_pymnt
0,0.443038,0.443038,0.450000,1.0,0.322430,0.239133,0.333333,0.323529,1.000000,0.666667,0.005455,0.0,0.02188,1.0000,1.0000,0.141176,0.011494,0.003617,0.043882,0.109195,0.0,1.0000,0.0,0.0,0.0,0.0,0.0,0.2,3.135739e-05,0.018790,0.051724,0.0020,9.742784e-03,0.114058,0.004744,0.221622,0.00275,0.030303,0.019231,0.009223,0.032628,0.135643,0.000004,0.168,0.031406,0.252011,0.028571,0.076923,0.0105,1.0000,0.0009,1.0000,0.111111,0.081967,0.127273,0.112676,0.03750,0.123288,0.085271,0.111111,0.141176,1.000000,0.504950,0.083333,0.011628,0.013949,0.007974,1.972047e-02,1.897842e-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.644258,0.443038,0.312178,0.312178,0.50,0.307692,0.307692,0.458333,0.317,0.445860,0.0,0.003819,0.0,0.0,0.296482,0.296482,0.158464,0.000571,0.000857,0.000400,0.000600,0.000571,0.000857,0.000400,0.000600,0.000833,0.001249,0.000583,0.000874,0.000009,0.000008,0.000012,0.000011,0.000009,0.000008,0.000012,0.000011,0.000028,0.000009,0.000038,0.000013,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20972.315627
1,0.348734,0.348734,0.356875,0.0,0.746885,0.325850,0.666667,0.647059,0.272727,0.666667,0.009092,1.0,0.01576,1.0000,1.0000,0.082353,0.011494,0.017896,0.096048,0.120690,0.0,1.0000,0.0,0.0,0.0,0.0,0.0,0.2,1.092592e-07,0.081764,0.017241,0.0059,5.844501e-07,0.000000,0.029207,0.464865,0.00613,0.030303,0.019231,0.088280,0.020777,0.258074,0.000004,0.130,0.017403,0.013405,0.015873,0.076923,0.0004,1.0000,0.0004,1.0000,0.111111,0.081967,0.090909,0.140845,0.02500,0.082192,0.116279,0.111111,0.082353,1.000000,0.670297,0.083333,0.011628,0.046930,0.015253,5.246581e-02,4

In [125]:
target_subsample

,investor_expect
0,0
1,0
2,0
3,0
4,0
...,...
250387,1
250388,1
250389,1
250390,1


In [126]:
target_subsample.value_counts()

investor_expect
0                  125196
1                  125196
dtype: int64

In [130]:
acc_subsample.to_csv("idea_3_downsampled.csv.gz",
                index=False,
                compression="gzip")

target_subsample.to_csv("idea_3_target_downsampled.csv.gz",
                        index=False,
                        compression="gzip")

In [ ]:
grade_cols = [col for col in acc_subsample.columns if 'grade' in col]

grade_cols

In [ ]:
acc_subsample_nogrades = acc_subsample.drop(grade_cols, axis=1)

acc_subsample_nogrades

In [ ]:
acc_subsample_nogrades.to_csv("acc_2_train_downsample_nogrades.csv.gz",
                              index=False,
                              compression="gzip")

# Viviann and Ali's Reduced Dataset

In [ ]:
# Random Forest

acc_VC_rf = copy.deepcopy(acc_subsample[['int_rate', 'dti', 'avg_cur_bal', 'revol_bal',
                                         'duration_of_credit_months', 'ratio_inc_funded',
                                         'ratio_redinc_loan', 'installment', 'annual_inc',
                                         'loan_amnt', 'open_acc', 'grouped_subgrade_12_345',
                                         'emp_length', ]])

acc_VC_gb = copy.deepcopy(acc_subsample[['grouped_subgrade_12_345', 'grade', 'term',
                                         'ratio_inc_funded', 'max_bal_bc', 'int_rate', 'fico_score',
                                         'dti', 'emp_length', 'home_ownership', 'revol_bal',
                                         'annual_inc', 'duration_of_credit_months']])

acc_VC_rf.to_csv("acc_VC_rf.csv.gz",
                 index=False,
                 compression="gzip")

acc_VC_gb.to_csv("acc_VC_gb.csv.gz",
                 index=False,
                 compression="gzip")

# Theodore's Reduced Dataset

In [ ]:
acc_TC = copy.deepcopy(acc_subsample[['annual_inc', 'annual_inc_joint', 'application_type', 'dti',
                                      'dti_joint', 'emp_length', 'duration_of_credit_months',
                                      'funded_amnt', 'fico_score', 'home_ownership', 'installment',
                                      'int_rate', 'loan_amnt', 'term', 'verification_status',
                                      'verification_status_joint', 'sec_app_fico_score',
                                      'sec_app_duration_of_credit_months', 'grade', 'sub_grade']])


acc_TC.to_csv("acc_TC.csv.gz",
              index=False,
              compression="gzip")

# Very Reduced Dataset

In [ ]:
cols_w_purpose = [col for col in acc_subsample.columns if 'purpose' in col]
acc_subsample[cols_w_purpose]

In [ ]:
cols_w_purpose = cols_w_purpose + ['annual_inc', 'fico_score', 'term', 'loan_amnt', 'home_ownership',
                                   'int_rate', 'all_util']

cols_w_purpose

In [ ]:
acc_vred = copy.deepcopy(acc_subsample[['annual_inc', 'fico_score', 'term', 'loan_amnt',
                                        'home_ownership', 'int_rate', 'all_util','dti',
                                        'total_pymnt']])


acc_vred.to_csv("acc_vred.csv.gz",
              index=False,
              compression="gzip")